In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import copy
from sqlalchemy import create_engine, MetaData

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

### 1. Xử lý dữ liệu

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("t2m_classification.xlsx", sheet_name='name_map').drop(columns=['group'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

In [3]:
#Đọc toàn bộ các file csv được xuất ra từ ami
total_items_dict = {}
# folder_path = '../ami_eod_data'
folder_path = 'D:\\t2m-project\\ami-data\\ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        total_items_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

#Tạo bảng tổng hớp tất cả các item
total_items_df = pd.DataFrame(list(total_items_dict.keys())).rename(columns={0:'item'})
total_items_df['len'] = total_items_df['item'].apply(lambda x: len(x))
total_items_df['last_2chars'] = total_items_df['item'].str[-2:]
total_items_df['third_last_char'] = total_items_df['item'].str[-3:-2]
total_items_df['first_4chars'] = total_items_df['item'].str[:4]

for item, df in total_items_dict.items():
    df = df.loc[df['date'] > 200302]
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d')
    total_items_dict[item] = df

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = total_items_df[total_items_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_name_df = total_items_df[(total_items_df['len']>3) & (total_items_df['len']!=6) & (total_items_df['len']<10) & 
                (total_items_df['third_last_char']!="F") & (total_items_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
                
stock_dict = {k:v.drop(['option'], axis=1) for k,v in total_items_dict.items() if k in stock_name_df['item'].tolist()}
index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in total_items_dict.items() if k in index_name_df['item'].tolist()}

#Lọc ra danh sách tên các cổ phiếu, index giao dịch tự doanh và nước ngoài
stock_name_td_df = total_items_df[(total_items_df['len']==6) & (total_items_df['last_2chars']=='TD')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
stock_name_nn_df = total_items_df[(total_items_df['len']==6) & (total_items_df['last_2chars']=='NN')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_td_nn_df = total_items_df[(total_items_df['len']>=10) & (total_items_df['first_4chars']!='VN30') & ((total_items_df['last_2chars']=='NN') | (total_items_df['last_2chars']=='TD'))].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)

stock_td_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in total_items_dict.items() if k in stock_name_td_df['item'].tolist()}
stock_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in total_items_dict.items() if k in stock_name_nn_df['item'].tolist()}
index_td_nn_dict = {k:v.drop(['high','low','cap','stock'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in total_items_dict.items() if k in index_td_nn_df['item'].tolist()}

for df in index_td_nn_dict.values():
    df['buy_volume'] = df['buy_volume']/1000
    df['sell_volume'] = -df['sell_volume']/1000
    df['buy_value'] = df['buy_value']/1000000000
    df['sell_value'] = -df['sell_value']/1000000000
    df['net_volume'] = df['buy_volume'] + df['sell_volume']
    df['net_value'] = df['buy_value'] + df['sell_value']

#Lọc ra danh sách các chỉ số phái sinh
index_name_fu_df = total_items_df[(total_items_df['len']>6) & (total_items_df['third_last_char']=="F")].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_dict = {k:v.drop(['cap'], axis=1).rename(columns={'option':'open_volume'})
                    for k,v in total_items_dict.items() if k in index_name_fu_df['item'].tolist()}

#Lọc ra danh sách các chỉ số phái sinh khối ngoại
index_name_fu_nn_df = total_items_df[(total_items_df['len']>9) & (total_items_df['first_4chars']=='VN30') & (total_items_df['last_2chars']=='NN')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in total_items_dict.items() if k in index_name_fu_nn_df['item'].tolist()}

#Lọc ra danh sách các chỉ số phái sinh tự doanh
index_name_fu_td_df = total_items_df[(total_items_df['len']>9) & (total_items_df['first_4chars']=='VN30') & (total_items_df['last_2chars']=='TD')].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_fu_td_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in total_items_dict.items() if k in index_name_fu_td_df['item'].tolist()}

#Tạo một date_series bao gồm khoảng ngày tính toán
date_series = pd.DataFrame(index_dict['VNINDEX']['date']).rename(columns={0:'date'})

In [4]:
start_time_am = dt.time(9, 00)
end_time_am = dt.time(11, 30)
start_time_pm = dt.time(13, 00)
end_time_pm = dt.time(15, 00)

def time_difference_in_minutes(time1, time2):
    delta1 = dt.timedelta(hours=time1.hour, minutes=time1.minute, seconds=time1.second)
    delta2 = dt.timedelta(hours=time2.hour, minutes=time2.minute, seconds=time2.second)
    diff = delta2 - delta1
    return diff.seconds // 60

def get_current_time(start_time_am, end_time_am, start_time_pm, end_time_pm):
    if (dt.datetime.now()).weekday() <= 4:
        current_time = dt.datetime.now().time()
        if current_time < start_time_am: current_time = end_time_pm
        elif (current_time >= start_time_am) & (current_time < end_time_am): current_time = current_time
        elif (current_time >= end_time_am) & (current_time < start_time_pm): current_time = end_time_am
        elif (current_time >= start_time_pm) & (current_time < end_time_pm): current_time = current_time
        elif current_time >= end_time_pm: current_time = end_time_pm
        return current_time
    if (dt.datetime.now()).weekday() > 4:
        return end_time_pm

current_time = get_current_time(start_time_am, end_time_am, start_time_pm, end_time_pm)

if current_time <= end_time_am:
    current_time_range = time_difference_in_minutes(start_time_am, current_time)
elif current_time >= start_time_pm:
    current_time_range = time_difference_in_minutes(start_time_am, current_time) - time_difference_in_minutes(end_time_am, start_time_pm)

# if current_time_range == 0:
#     time_percent = 1
# elif date_series.iloc[0].item().strftime("%Y-%m-%d") != datetime.now().date().strftime("%Y-%m-%d"):
#     time_percent = 1
# else:
#     time_percent = current_time_range/270

time_percent = 0.8518518518518519

In [5]:
stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in stock_dict.items()}

stock_dict = {
    key: df.assign(

        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),
        ma20_V=df['volume'].rolling(window=20, min_periods=1).mean().shift(1),
        ma60_V=df['volume'].rolling(window=60, min_periods=1).mean().shift(1),
        ma120_V=df['volume'].rolling(window=120, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in stock_dict.items()
}

stock_dict = {
    key: df.assign(

        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)

    )
    for key, df in stock_dict.items()
}
stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in stock_dict.items()}

#Tính hệ số thanh khoản và đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in stock_dict.values():
    df['liquid_ratio'] = df['volume'] / (df['ma5_V'])
    df['liquid_ratio'].iloc[0] = df['volume'].iloc[0] / ((df['ma5_V']).iloc[0]*time_percent)
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

#Xoá các cổ phiếu chưa có giao dịch trong ngày
delete_stock = []
for stock, df in stock_dict.items():
    if date_series['date'].iloc[0] != df['date'].iloc[0]:
        delete_stock.append(stock)
for stock in delete_stock:
    stock_dict.pop(stock)

#Xoá các cổ phiếu có giá bị lỗi bằng 0
delete_stock = []
for stock, df in stock_dict.items():
    if df['close'].min() == 0:
        delete_stock.append(stock)
for stock in delete_stock:
    stock_dict.pop(stock)

In [6]:
stock_classification = pd.read_excel('t2m_classification.xlsx')
stock_classification = stock_classification[stock_classification['stock'].isin(list(stock_dict.keys()))]

#Tạo các mảng dữ liệu vốn hoá và giá của phiên T0
price_arr = []
cap_arr = []
for stock, df in stock_dict.items():
    price_arr.append(df['close'].iloc[0].item())
    cap_arr.append(df['cap'].iloc[0].item())

#Tạo bảng chia nhóm vốn hoá
vonhoa_classification_df = stock_classification.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
    'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
               ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
               else
    ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
              ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
              else
    ('large' if x['cap']>=100*cap_coef
               else 'penny'
)), axis=1)

stock_classification = pd.concat([stock_classification, vonhoa_classification_df['marketcap_group']], axis=1)

all_stock = {}
all_stock['all_stock'] = {key: value for key, value in stock_dict.items()}

industry_name = {}
for name in np.sort(stock_classification['industry_name'].unique()):
    industry_name[name] = {key: value for key, value in stock_dict.items() if key in stock_classification[stock_classification['industry_name'] == name]['stock'].to_list()}

industry_perform = {}
for name in np.sort(stock_classification['industry_perform'].unique()):
    industry_perform[name] = {key: value for key, value in stock_dict.items() if key in stock_classification[stock_classification['industry_perform'] == name]['stock'].to_list()}

marketcap_group = {}
for name in ['large', 'mid', 'small', 'penny']:
    marketcap_group[name] = {key: value for key, value in stock_dict.items() if key in stock_classification[stock_classification['marketcap_group'] == name]['stock'].to_list()}

In [7]:
stock_group_name = ['all_stock', 'industry_name', 'industry_perform', 'marketcap_group']
stock_group = [all_stock, industry_name, industry_perform, marketcap_group]

unique_dates = date_series['date'].tolist()

#Tạo các bảng trend MS để vẽ biểu đồ
for name, group in zip(stock_group_name, stock_group):

    globals()[f"{name}_trend"] = {key: {} for key in group.keys()}

    for group_name in group.keys():

        #Tạo dict chứa các cổ phiếu thuộc cùng nhóm
        trend_dict = {'trend_5p': None, 'trend_20p': None,'trend_60p': None,'trend_120p': None,'trend_240p': None, 'trend_480p': None}
        for trend in trend_dict.keys():
            trend_dict[trend] = pd.DataFrame(unique_dates, columns=['date'])
            for stock in group[group_name].keys():
                trend_dict[trend][stock] = group[group_name][stock][trend]
            trend_dict[trend].fillna(value=0, inplace=True)
            trend_dict[trend]['sum'] = trend_dict[trend].iloc[:, 1:len(group[group_name])+1].apply(sum, axis=1)
            trend_dict[trend]['percent'] = trend_dict[trend]['sum'] / len(group[group_name])

        #Tạo bảng dữ liệu theo ngày
        globals()[f"{name}_trend"][group_name] = pd.DataFrame(unique_dates, columns=['date'])
        for trend in trend_dict.keys():
            globals()[f"{name}_trend"][group_name][trend] = trend_dict[trend]['percent']

trend_group = [all_stock_trend, industry_name_trend, industry_perform_trend, marketcap_group_trend]

In [8]:
all_industry_index_trend = pd.DataFrame()
for index_name, df in industry_name_trend.items():
    temp_df = df.copy()
    temp_df['id'] = index_name
    all_industry_index_trend = pd.concat([all_industry_index_trend, temp_df.iloc[:60]], axis=0)
all_industry_index_trend['id'] = all_industry_index_trend['id'].map(name_map_dict)

all_industry_perform_index_trend = pd.DataFrame()
for index_name, df in industry_perform_trend.items():
    temp_df = df.copy()
    temp_df['id'] = index_name
    all_industry_perform_index_trend = pd.concat([all_industry_perform_index_trend, temp_df.iloc[:60]], axis=0)
all_industry_perform_index_trend['id'] = all_industry_perform_index_trend['id'].map(name_map_dict)

all_marketcap_group_index_trend = pd.DataFrame()
for index_name, df in marketcap_group_trend.items():
    temp_df = df.copy()
    temp_df['id'] = index_name
    all_marketcap_group_index_trend = pd.concat([all_marketcap_group_index_trend, temp_df.iloc[:60]], axis=0)
all_marketcap_group_index_trend['id'] = all_marketcap_group_index_trend['id'].map(name_map_dict)
all_marketcap_group_index_trend['order'] = all_marketcap_group_index_trend['id'].map(name_map_dict)

### 2. Tính điểm dòng tiền

In [9]:
stock_dict_trimmed = {k: v.iloc[:100].reset_index(drop=True) for k, v in stock_dict.items()}

In [10]:
def score_calculation(df):
    try:
        result = (((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low']) *
                  abs((df['close'] - df['close_prev'])) / df['close_prev'] *
                  (df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V'])) * 100 \
                  + ((df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V']))/100
        result.fillna(0, inplace=True)
        return result
    except ZeroDivisionError:
        # return 0
        return ((df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V']))/100

stock_score_dict = {}
for stock in stock_dict_trimmed.keys():
    stock_score_dict[stock] = stock_dict_trimmed[stock][['stock', 'date', 'high', 'low', 'close', 'volume','liquid_ratio','cap', 'ma5_V', 'ma20_V','ma60_V','ma120_V','ma5']]
    stock_score_dict[stock]['ma5_prev'] = stock_score_dict[stock]['ma5'].shift(-1)
    stock_score_dict[stock]['close_prev'] = stock_score_dict[stock]['close'].shift(-1)
    
    stock_score_dict[stock]['raw_score'] = score_calculation(stock_score_dict[stock])
    stock_score_dict[stock]['raw_score'].iloc[0] = stock_score_dict[stock]['raw_score'].iloc[0].item()/time_percent

    stock_score_dict[stock].sort_values('date', ascending=True, inplace=True)
    stock_score_dict[stock]['highest_price'] = stock_score_dict[stock]['close'].rolling(window=40, min_periods=1).max()
    stock_score_dict[stock]['lowest_volume20'] = stock_score_dict[stock]['volume'].rolling(window=20, min_periods=1).min().shift(1)
    stock_score_dict[stock]['lowest_volume5'] = stock_score_dict[stock]['volume'].rolling(window=5, min_periods=1).min().shift(1)
    stock_score_dict[stock].sort_values('date', ascending=False, inplace=True)

def get_total_cap(df, group, stock_dict_trimmed):
    unique_groups = df[group].unique()
    total_cap_dict = {}
    for unique_group in unique_groups:
        total_cap = date_series.copy()
        keys = df[df[group] == unique_group]['stock'].tolist()
        for key in keys:
            total_cap[key] = stock_dict_trimmed[key]['cap']
        total_cap['total'] = total_cap.iloc[:, 1:].sum(axis=1)
        total_cap_dict[unique_group] = total_cap['total']
    return total_cap_dict

total_cap_industry_dict = get_total_cap(stock_classification, 'industry_name', stock_dict_trimmed)
total_cap_marketcap_dict = get_total_cap(stock_classification, 'marketcap_group', stock_dict_trimmed)

for stock in stock_score_dict.keys():
    stock_score_dict[stock]['total_cap_industry'] = total_cap_industry_dict[stock_classification[stock_classification['stock'] == stock]['industry_name'].item()]
    stock_score_dict[stock]['total_cap_marketcap'] = total_cap_marketcap_dict[stock_classification[stock_classification['stock'] == stock]['marketcap_group'].item()]

for stock in stock_score_dict.keys():
    stock_score_dict[stock]['industry_cont'] = stock_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_industry'], axis=1)
    stock_score_dict[stock]['marketcap_cont'] = stock_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_marketcap'], axis=1)

In [11]:
stock_score_dict_filtered = {}
for stock, df in stock_score_dict.items():
    if  (df['ma5_V'][0] >= 50000) & \
        (df['ma20_V'][0] >= 50000) & \
        (df['ma60_V'][0] >= 50000) & \
        (df['ma120_V'][0] >= 50000) & \
        (df['lowest_volume20'][0] > 0) & \
        (df['lowest_volume5'][0] >= 50000) & \
        (df['close'][0] > df['highest_price'][0]*0.382):

        stock_score_dict_filtered[stock] = df[['stock', 'date', 'close', 'volume', 'liquid_ratio', 'raw_score', 'industry_cont', 'marketcap_cont','cap']]

stock_classification_filtered = stock_classification[stock_classification['stock'].isin(stock_score_dict_filtered.keys())]

In [12]:
temp_industry_score_dict = {}
for nganh in stock_classification_filtered['industry_name'].unique():
    industry_score = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock'].tolist():
        industry_score[stock] = stock_score_dict_filtered[stock]['industry_cont']
    temp_industry_score_dict[nganh] = industry_score.iloc[:, 1:].sum(axis=1)

temp_marketcap_score_dict = {}
for nganh in stock_classification_filtered['marketcap_group'].unique():
    marketcap_score = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==nganh]['stock'].tolist():
        marketcap_score[stock] = stock_score_dict_filtered[stock]['marketcap_cont']
    temp_marketcap_score_dict[nganh] = marketcap_score.iloc[:, 1:].sum(axis=1)

for stock in stock_score_dict_filtered.keys():
    nganh = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    marketcap = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    stock_score_dict_filtered[stock]['industry_score'] = temp_industry_score_dict[nganh]
    stock_score_dict_filtered[stock]['marketcap_score'] = temp_marketcap_score_dict[marketcap]
    stock_score_dict_filtered[stock]['t0_score'] = stock_score_dict_filtered[stock].apply(lambda x: x['raw_score']*0.7+x['industry_score']*0.2+x['marketcap_score']*0.1, axis=1)

    stock_score_dict_filtered[stock].sort_values('date', ascending=True, inplace=True)
    stock_score_dict_filtered[stock]['t5_score'] = stock_score_dict_filtered[stock]['t0_score'].rolling(window=5, min_periods=1).mean()
    stock_score_dict_filtered[stock].sort_values('date', ascending=False, inplace=True)

    stock_score_dict_filtered[stock]['industry_name'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    stock_score_dict_filtered[stock]['industry_perform'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    stock_score_dict_filtered[stock]['stock_perform'] = stock_classification_filtered[stock_classification_filtered['stock'] == stock]['stock_perform'].item()
    stock_score_dict_filtered[stock]['marketcap_group'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()
    stock_score_dict_filtered[stock]['t2m_select'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['t2m_select'].item()

In [13]:
group_score = date_series.copy()
ranking_group = date_series.copy()

#Xếp hạng T5
for stock in stock_score_dict_filtered.keys():
    group_score[stock] = stock_score_dict_filtered[stock]['t5_score']
    group_score.fillna(0, inplace=True)
    ranking_group[stock] = 0
ranking_group = group_score.iloc[:,1:].rank(ascending=False, method='min', axis=1)

for stock, df in stock_score_dict_filtered.items():
    df['price_change'] = df['close'][::-1].pct_change()[::-1]
    df['rank'] = ranking_group[stock]
    df['rank_prev'] = df['rank'].shift(-1)
    df['rank_change'] = df['rank_prev'] - df['rank']

#Xếp hạng T0
for stock in stock_score_dict_filtered.keys():
    group_score[stock] = stock_score_dict_filtered[stock]['t0_score']
    group_score.fillna(0, inplace=True)
    ranking_group[stock] = 0
ranking_group = group_score.iloc[:,1:].rank(ascending=False, method='min', axis=1)

for stock, df in stock_score_dict_filtered.items():
    df['rank_t0'] = ranking_group[stock]
    df['rank_t0_prev'] = df['rank_t0'].shift(-1)

#Check xem xếp hạng T0 nằm trong top 10% hay không
for stock, df in stock_score_dict_filtered.items():
    df['top_check'] = df['rank_t0'].apply(lambda x: 1 if x <= len(stock_classification_filtered)*0.1 else 0)
    df['top_count'] = df['top_check'][::-1].rolling(window=20).sum()[::-1]

# Tạo điểm số cuối cùng
final_score_dict = {k: v.iloc[:20].drop(
    columns=['raw_score', 'industry_cont', 'marketcap_cont',
             'industry_score', 'marketcap_score', 'rank_t0_prev', 'rank_prev', 'top_check'])
    for k, v in stock_score_dict_filtered.items()}

### 3. Các chỉ báo kĩ thuật

#### Các hàm tính toán chỉ số kĩ thuật

In [14]:
def find_peak_and_bottom(df):
    df_copy = df.copy()

    df_copy['close_p1'] = df_copy['close'].shift(-1)
    df_copy['close_p2'] = df_copy['close'].shift(-2)
    df_copy['close_f1'] = df_copy['close'].shift(1)
    df_copy['close_f2'] = df_copy['close'].shift(2)

    df_copy['high_p1'] = df_copy['high'].shift(-1)
    df_copy['high_p2'] = df_copy['high'].shift(-2)
    df_copy['high_f1'] = df_copy['high'].shift(1)
    df_copy['high_f2'] = df_copy['high'].shift(2)

    df_copy['low_p1'] = df_copy['low'].shift(-1)
    df_copy['low_p2'] = df_copy['low'].shift(-2)
    df_copy['low_f1'] = df_copy['low'].shift(1)
    df_copy['low_f2'] = df_copy['low'].shift(2)

    df_copy['peak'] = ((df_copy['high']>df_copy['close_p1']) & (df_copy['high']>df_copy['close_p2']) &
                        (df_copy['high']>df_copy['close_f1']) & (df_copy['high']>df_copy['close_f2']) &
                        (df_copy['high'] == df_copy[['high', 'high_p1', 'high_p2', 'high_f1', 'high_f2']].max(axis=1))).astype(int)

    df_copy['bottom'] = ((df_copy['low']<df_copy['close_p1']) & (df_copy['low']<df_copy['close_p2']) &
                        (df_copy['low']<df_copy['close_f2']) & (df_copy['low']<df_copy['close_f2']) &
                        (df_copy['low'] == df_copy[['low', 'low_p1', 'low_p2', 'low_f1', 'low_f2']].min(axis=1))).astype(int)

    return df_copy['peak'], df_copy['bottom']

In [15]:
def calculate_rsi(data):

    data = data.sort_values('date')
    data['change'] = data['close'].diff()
    data['gain'] = data['change'].apply(lambda x: x if x > 0 else 0)
    data['loss'] = data['change'].apply(lambda x: -x if x < 0 else 0)

    # Tính trung bình cử động (EMA) cho gain và loss
    alpha = 1/14
    data['avg_gain'] = data['gain'].ewm(alpha=alpha, adjust=False).mean()
    data['avg_loss'] = data['loss'].ewm(alpha=alpha, adjust=False).mean()

    data['rs'] = data['avg_gain'] / data['avg_loss']
    data['rsi'] = 100 - (100 / (1 + data['rs']))

    return data['rsi']

def find_peak_and_bottom_rsi(df):
    rsi_values = df['rsi'].values
    peaks = [0] * len(rsi_values)
    bottoms= [0] * len(rsi_values)

    for i in range(1, len(rsi_values) - 1):
        # Đỉnh
        if rsi_values[i] > rsi_values[i - 1] and rsi_values[i] > rsi_values[i + 1]:
            peaks[i] = 1
        else:
            peaks[i] = 0
        # Đáy
        if rsi_values[i] < rsi_values[i - 1] and rsi_values[i] < rsi_values[i + 1]:
            bottoms[i] = 1
        else:
            bottoms[i] = 0

    return peaks, bottoms

In [16]:
def calculate_ta_df(price_df):
    ta_df = price_df[['stock','date','open','high','low','close','volume']]
    ta_df['month'] = ta_df['date'].dt.to_period('M').astype(str)
    ta_df['quarter'] = ta_df['date'].dt.to_period('Q').astype(str)
    ta_df['year'] = ta_df['date'].dt.to_period('Y').astype(str)
    return ta_df

In [17]:
def calculate_candle_ta_df(ta_df,input_type):
    candle_ta_df = calculate_ta_df(ta_df)
    temp_df = copy.deepcopy(candle_ta_df)
    temp_df['peak'] = find_peak_and_bottom(temp_df)[0]
    temp_df['bottom'] = find_peak_and_bottom(temp_df)[1]
    ta_df_peak = temp_df[temp_df['peak']==1].drop(['peak','bottom'], axis=1)
    ta_df_bottom = temp_df[temp_df['bottom']==1].drop(['peak','bottom'], axis=1)

    ta_df_peak['month'] = temp_df['date'].dt.to_period('M').astype(str)
    ta_df_peak['quarter'] = temp_df['date'].dt.to_period('Q').astype(str)
    ta_df_peak['year'] = temp_df['date'].dt.to_period('Y').astype(str)

    ta_df_bottom['month'] = temp_df['date'].dt.to_period('M').astype(str)
    ta_df_bottom['quarter'] = temp_df['date'].dt.to_period('Q').astype(str)
    ta_df_bottom['year'] = temp_df['date'].dt.to_period('Y').astype(str)
    
    candle_ta_df['month_bottom'] = candle_ta_df[candle_ta_df['month'] == candle_ta_df['month'].unique()[0]]['low'].min()
    candle_ta_df['quarter_bottom'] = candle_ta_df[candle_ta_df['quarter'] == candle_ta_df['quarter'].unique()[0]]['low'].min()
    candle_ta_df['year_bottom'] = candle_ta_df[candle_ta_df['year'] == candle_ta_df['year'].unique()[0]]['low'].min()

    candle_ta_df['month_peak'] = candle_ta_df[candle_ta_df['month'] == candle_ta_df['month'].unique()[0]]['high'].max()
    candle_ta_df['quarter_peak'] = candle_ta_df[candle_ta_df['quarter'] == candle_ta_df['quarter'].unique()[0]]['high'].max()
    candle_ta_df['year_peak'] = candle_ta_df[candle_ta_df['year'] == candle_ta_df['year'].unique()[0]]['high'].max()

    candle_ta_df['month_open'] = candle_ta_df[candle_ta_df['month'] == candle_ta_df['month'].unique()[0]]['open'].iloc[-1]
    candle_ta_df['quarter_open'] = candle_ta_df[candle_ta_df['quarter'] == candle_ta_df['quarter'].unique()[0]]['open'].iloc[-1]
    candle_ta_df['year_open'] = candle_ta_df[candle_ta_df['year'] == candle_ta_df['year'].unique()[0]]['open'].iloc[-1]

    if input_type == 'stock':

        candle_ta_df['from_month_peak'] = (candle_ta_df['close'] - candle_ta_df['month_peak'])/candle_ta_df['month_peak']
        candle_ta_df['from_quarter_peak'] = (candle_ta_df['close'] - candle_ta_df['quarter_peak'])/candle_ta_df['quarter_peak']
        candle_ta_df['from_year_peak'] = (candle_ta_df['close'] - candle_ta_df['year_peak'])/candle_ta_df['year_peak']

        candle_ta_df['from_month_bottom'] = (candle_ta_df['close'] - candle_ta_df['month_bottom'])/candle_ta_df['month_bottom']
        candle_ta_df['from_quarter_bottom'] = (candle_ta_df['close'] - candle_ta_df['quarter_bottom'])/candle_ta_df['quarter_bottom']
        candle_ta_df['from_year_bottom'] = (candle_ta_df['close'] - candle_ta_df['year_bottom'])/candle_ta_df['year_bottom']

        candle_ta_df['from_month_open'] = (candle_ta_df['close'] - candle_ta_df['month_open'])/candle_ta_df['month_open']
        candle_ta_df['from_quarter_open'] = (candle_ta_df['close'] - candle_ta_df['quarter_open'])/candle_ta_df['quarter_open']
        candle_ta_df['from_year_open'] = (candle_ta_df['close'] - candle_ta_df['year_open'])/candle_ta_df['year_open']

    if input_type == 'index':

        candle_ta_df['from_month_peak'] = (candle_ta_df['close'] - candle_ta_df['month_peak'])
        candle_ta_df['from_quarter_peak'] = (candle_ta_df['close'] - candle_ta_df['quarter_peak'])
        candle_ta_df['from_year_peak'] = (candle_ta_df['close'] - candle_ta_df['year_peak'])

        candle_ta_df['from_month_bottom'] = (candle_ta_df['close'] - candle_ta_df['month_bottom'])
        candle_ta_df['from_quarter_bottom'] = (candle_ta_df['close'] - candle_ta_df['quarter_bottom'])
        candle_ta_df['from_year_bottom'] = (candle_ta_df['close'] - candle_ta_df['year_bottom'])

        candle_ta_df['from_month_open'] = (candle_ta_df['close'] - candle_ta_df['month_open'])
        candle_ta_df['from_quarter_open'] = (candle_ta_df['close'] - candle_ta_df['quarter_open'])
        candle_ta_df['from_year_open'] = (candle_ta_df['close'] - candle_ta_df['year_open'])

    return candle_ta_df

In [18]:
def calculate_fibo_ta_df(ta_df, input_type):
    fibo_ta_df = calculate_ta_df(ta_df)

    fibo_ta_df['month_peak'] = fibo_ta_df[fibo_ta_df['month'].isin(fibo_ta_df['month'].unique()[:2].tolist())]['high'].max()
    fibo_ta_df['quarter_peak'] = fibo_ta_df[fibo_ta_df['quarter'].isin(fibo_ta_df['quarter'].unique()[:2].tolist())]['high'].max()
    fibo_ta_df['year_peak'] = fibo_ta_df[fibo_ta_df['year'].isin(fibo_ta_df['year'].unique()[:2].tolist())]['high'].max()

    fibo_ta_df['month_bottom'] = fibo_ta_df[fibo_ta_df['month'].isin(fibo_ta_df['month'].unique()[:2].tolist())]['low'].min()
    fibo_ta_df['quarter_bottom'] = fibo_ta_df[fibo_ta_df['quarter'].isin(fibo_ta_df['quarter'].unique()[:2].tolist())]['low'].min()
    fibo_ta_df['year_bottom'] = fibo_ta_df[fibo_ta_df['year'].isin(fibo_ta_df['year'].unique()[:2].tolist())]['low'].min()

    month_trend = (fibo_ta_df[fibo_ta_df['high']==fibo_ta_df['month_peak']]['date'].iloc[0] - fibo_ta_df[fibo_ta_df['low']==fibo_ta_df['month_bottom']]['date'].iloc[0]).days
    quarter_trend = (fibo_ta_df[fibo_ta_df['high']==fibo_ta_df['quarter_peak']]['date'].iloc[0] - fibo_ta_df[fibo_ta_df['low']==fibo_ta_df['quarter_bottom']]['date'].iloc[0]).days
    year_trend = (fibo_ta_df[fibo_ta_df['high']==fibo_ta_df['year_peak']]['date'].iloc[0] - fibo_ta_df[fibo_ta_df['low']==fibo_ta_df['year_bottom']]['date'].iloc[0]).days

    if month_trend >= 0:
        fibo_ta_df['month_trend'] = 'Tăng'
    else:
        fibo_ta_df['month_trend'] = 'Giảm'
        
    fibo_ta_df['month_fibo_382'] = fibo_ta_df['month_peak'] - (fibo_ta_df['month_peak'] - fibo_ta_df['month_bottom'])*0.382
    fibo_ta_df['month_fibo_500'] = fibo_ta_df['month_peak'] - (fibo_ta_df['month_peak'] - fibo_ta_df['month_bottom'])*0.5
    fibo_ta_df['month_fibo_618'] = fibo_ta_df['month_peak'] - (fibo_ta_df['month_peak'] - fibo_ta_df['month_bottom'])*0.618

    if quarter_trend >= 0:
        fibo_ta_df['quarter_trend'] = 'Tăng'
    else:
        fibo_ta_df['quarter_trend'] = 'Tăng'

    fibo_ta_df['quarter_fibo_382'] = fibo_ta_df['quarter_peak'] - (fibo_ta_df['quarter_peak'] - fibo_ta_df['quarter_bottom'])*0.382
    fibo_ta_df['quarter_fibo_500'] = fibo_ta_df['quarter_peak'] - (fibo_ta_df['quarter_peak'] - fibo_ta_df['quarter_bottom'])*0.5
    fibo_ta_df['quarter_fibo_618'] = fibo_ta_df['quarter_peak'] - (fibo_ta_df['quarter_peak'] - fibo_ta_df['quarter_bottom'])*0.618

    if year_trend >= 0:
        fibo_ta_df['year_trend'] = 'Tăng'
    else:
        fibo_ta_df['year_trend'] = 'Giảm'

    fibo_ta_df['year_fibo_382'] = fibo_ta_df['year_peak'] - (fibo_ta_df['year_peak'] - fibo_ta_df['year_bottom'])*0.382
    fibo_ta_df['year_fibo_500'] = fibo_ta_df['year_peak'] - (fibo_ta_df['year_peak'] - fibo_ta_df['year_bottom'])*0.5
    fibo_ta_df['year_fibo_618'] = fibo_ta_df['year_peak'] - (fibo_ta_df['year_peak'] - fibo_ta_df['year_bottom'])*0.618

    if input_type == 'stock':

        fibo_ta_df['from_month_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_382'])/abs(fibo_ta_df['month_fibo_382'])
        fibo_ta_df['from_month_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_500'])/abs(fibo_ta_df['month_fibo_500'])
        fibo_ta_df['from_month_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_618'])/abs(fibo_ta_df['month_fibo_618'])

        fibo_ta_df['from_quarter_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_382'])/abs(fibo_ta_df['quarter_fibo_382'])
        fibo_ta_df['from_quarter_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_500'])/abs(fibo_ta_df['quarter_fibo_500'])
        fibo_ta_df['from_quarter_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_618'])/abs(fibo_ta_df['quarter_fibo_618'])

        fibo_ta_df['from_year_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_382'])/abs(fibo_ta_df['year_fibo_382'])
        fibo_ta_df['from_year_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_500'])/abs(fibo_ta_df['year_fibo_500'])
        fibo_ta_df['from_year_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_618'])/abs(fibo_ta_df['year_fibo_618'])

    if input_type == 'index':
            
        fibo_ta_df['from_month_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_382'])
        fibo_ta_df['from_month_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_500'])
        fibo_ta_df['from_month_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['month_fibo_618'])

        fibo_ta_df['from_quarter_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_382'])
        fibo_ta_df['from_quarter_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_500'])
        fibo_ta_df['from_quarter_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['quarter_fibo_618'])

        fibo_ta_df['from_year_fibo_382'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_382'])
        fibo_ta_df['from_year_fibo_500'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_500'])
        fibo_ta_df['from_year_fibo_618'] = (fibo_ta_df['close'] - fibo_ta_df['year_fibo_618'])

    return fibo_ta_df

In [19]:
def calculate_pivot_ta_df(ta_df, input_type):
    pivot_ta_df = calculate_ta_df(ta_df)

    try: pivot_ta_df['month_peak'] = pivot_ta_df[pivot_ta_df['month']==pivot_ta_df['month'].unique()[1]]['high'].max()
    except: pivot_ta_df['month_peak'] = None
    try: pivot_ta_df['quarter_peak'] = pivot_ta_df[pivot_ta_df['quarter']==pivot_ta_df['quarter'].unique()[1]]['high'].max()
    except: pivot_ta_df['quarter_peak'] = None
    try: pivot_ta_df['year_peak'] = pivot_ta_df[pivot_ta_df['year']==pivot_ta_df['year'].unique()[1]]['high'].max()
    except: pivot_ta_df['year_peak'] = None

    try: pivot_ta_df['month_bottom'] = pivot_ta_df[pivot_ta_df['month']==pivot_ta_df['month'].unique()[1]]['low'].min()
    except: pivot_ta_df['month_bottom'] = None
    try: pivot_ta_df['quarter_bottom'] = pivot_ta_df[pivot_ta_df['quarter']==pivot_ta_df['quarter'].unique()[1]]['low'].min()
    except: pivot_ta_df['quarter_bottom'] = None
    try: pivot_ta_df['year_bottom'] = pivot_ta_df[pivot_ta_df['year']==pivot_ta_df['year'].unique()[1]]['low'].min()
    except: pivot_ta_df['year_bottom'] = None

    try: pivot_ta_df['month_close'] = pivot_ta_df[pivot_ta_df['month']==pivot_ta_df['month'].unique()[1]]['close'].iloc[0]
    except: pivot_ta_df['month_close'] = None
    try: pivot_ta_df['quarter_close'] = pivot_ta_df[pivot_ta_df['quarter']==pivot_ta_df['quarter'].unique()[1]]['close'].iloc[0]
    except: pivot_ta_df['quarter_close'] = None
    try: pivot_ta_df['year_close'] = pivot_ta_df[pivot_ta_df['year']==pivot_ta_df['year'].unique()[1]]['close'].iloc[0]
    except: pivot_ta_df['year_close'] = None

    pivot_ta_df['month_pivot_p'] = (pivot_ta_df['month_peak'] + pivot_ta_df['month_bottom'] + pivot_ta_df['month_close'])/3
    pivot_ta_df['quarter_pivot_p'] = (pivot_ta_df['quarter_peak'] + pivot_ta_df['quarter_bottom'] + pivot_ta_df['quarter_close'])/3
    pivot_ta_df['year_pivot_p'] = (pivot_ta_df['year_peak'] + pivot_ta_df['year_bottom'] + pivot_ta_df['year_close'])/3

    if input_type == 'index':
        pivot_ta_df['from_month_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['month_pivot_p'])
        pivot_ta_df['from_quarter_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['quarter_pivot_p'])
        pivot_ta_df['from_year_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['year_pivot_p'])

    if input_type == 'stock':
        pivot_ta_df['from_month_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['month_pivot_p'])/abs(pivot_ta_df['month_pivot_p'])
        pivot_ta_df['from_quarter_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['quarter_pivot_p'])/abs(pivot_ta_df['quarter_pivot_p'])
        pivot_ta_df['from_year_pivot_p'] = (pivot_ta_df['close'] - pivot_ta_df['year_pivot_p'])/abs(pivot_ta_df['year_pivot_p'])

    return pivot_ta_df

In [20]:
def calculate_ma_ta_df(ta_df,input_type):

    ma_ta_df = calculate_ta_df(ta_df)
    ma_ta_df['ma5'] = ma_ta_df['close'][::-1].rolling(window=5, min_periods=1).mean()[::-1]
    ma_ta_df['ma20'] = ma_ta_df['close'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
    ma_ta_df['ma60'] = ma_ta_df['close'][::-1].rolling(window=60, min_periods=1).mean()[::-1]
    ma_ta_df['ma120'] = ma_ta_df['close'][::-1].rolling(window=120, min_periods=1).mean()[::-1]
    ma_ta_df['ma240'] = ma_ta_df['close'][::-1].rolling(window=240, min_periods=1).mean()[::-1]
    ma_ta_df['ma480'] = ma_ta_df['close'][::-1].rolling(window=480, min_periods=1).mean()[::-1]

    if input_type == 'stock':

        ma_ta_df['from_ma5'] = (ma_ta_df['close'] - ma_ta_df['ma5'])/ma_ta_df['ma5']
        ma_ta_df['from_ma20'] = (ma_ta_df['close'] - ma_ta_df['ma20'])/ma_ta_df['ma20']
        ma_ta_df['from_ma60'] = (ma_ta_df['close'] - ma_ta_df['ma60'])/ma_ta_df['ma60']
        ma_ta_df['from_ma120'] = (ma_ta_df['close'] - ma_ta_df['ma120'])/ma_ta_df['ma120']
        ma_ta_df['from_ma240'] = (ma_ta_df['close'] - ma_ta_df['ma240'])/ma_ta_df['ma240']
        ma_ta_df['from_ma480'] = (ma_ta_df['close'] - ma_ta_df['ma480'])/ma_ta_df['ma480']

    if input_type == 'index':

        ma_ta_df['from_ma5'] = (ma_ta_df['close'] - ma_ta_df['ma5'])
        ma_ta_df['from_ma20'] = (ma_ta_df['close'] - ma_ta_df['ma20'])
        ma_ta_df['from_ma60'] = (ma_ta_df['close'] - ma_ta_df['ma60'])
        ma_ta_df['from_ma120'] = (ma_ta_df['close'] - ma_ta_df['ma120'])
        ma_ta_df['from_ma240'] = (ma_ta_df['close'] - ma_ta_df['ma240'])
        ma_ta_df['from_ma480'] = (ma_ta_df['close'] - ma_ta_df['ma480'])

    return ma_ta_df

In [21]:
def calculate_rsi_ta_df(ta_df):
    rsi_ta_df = calculate_ta_df(ta_df)

    df = rsi_ta_df
    df['percent_change'] = df['close'][::-1].pct_change()[::-1]
    df['value_change'] = df['close'][::-1].diff(1)[::-1]
    df['rsi'] = calculate_rsi(df)
    df['peak'] = find_peak_and_bottom_rsi(df)[0]
    df['bottom'] = find_peak_and_bottom_rsi(df)[1]

    df['prev_rsi_peak'] = df[df['peak']==1].iloc[0]['rsi']
    df['prev_rsi_bottom'] = df[df['bottom']==1].iloc[0]['rsi']
    df['prev_price_peak'] = df[df['peak']==1].iloc[0]['high']
    df['prev_price_bottom'] = df[df['bottom']==1].iloc[0]['low']

    df['month_trend_rsi'] = ['Tăng' if curr >= prev*1.08 else ('Giảm' if curr <= prev*0.92 else 'Side Way') for curr, prev in zip(df['close'], df['close'].shift(-20))]
    df['month_peak_rsi'] = df[df['peak']==1]['rsi'][::-1].rolling(window=int(df['peak'][::-1].rolling(window=20).sum()[::-1][0])).max()[::-1]
    df['month_peak_rsi'] = df['month_peak_rsi'][::-1].ffill()[::-1]
    df['month_bottom_rsi'] = df[df['bottom']==1]['rsi'][::-1].rolling(window=int(df['bottom'][::-1].rolling(window=20).sum()[::-1][0])).min()[::-1]
    df['month_bottom_rsi'] = df['month_bottom_rsi'][::-1].ffill()[::-1]
    try:
        df['month_peak_price'] = df[df['rsi']==df['month_peak_rsi'].iloc[0].item()]['high'].item()
    except:
        df['month_peak_rsi'] = df['rsi'].iloc[0].item()
        df['month_peak_price'] = df['close'].iloc[0].item()

    try:
        df['month_bottom_price'] = df[df['rsi']==df['month_bottom_rsi'].iloc[0].item()]['low'].item()
    except:
        df['month_bottom_rsi'] = df['rsi'].iloc[0].item()
        df['month_bottom_price'] = df['close'].iloc[0].item()
    df['rsi_state'] = df.apply(lambda x: 'Phân kì âm' if (x['month_peak_rsi']>x['prev_rsi_peak']) & (x['month_peak_price']<x['prev_price_peak']) & (x['month_trend_rsi']=='Tăng') else 
                                                ('Phân kì dương' if (x['month_bottom_rsi']<x['prev_rsi_bottom']) & (x['month_bottom_price']>x['prev_price_bottom']) & (x['month_trend_rsi']=='Giảm') else ''), axis=1)
    return rsi_ta_df[['stock','date','close','rsi','rsi_state']].iloc[:1]

#### Các hàm concat bẳng hoàn chỉnh

In [22]:
def transform_ta_df(ta_df,ta_name):
    df_list = []
    for time_frame in ['month','quarter','year']:
        coef_2 = 0
        if ta_name == 'candle':
            df = ta_df[['stock',f'{time_frame}_open',f'{time_frame}_peak',f'{time_frame}_bottom',f'from_{time_frame}_open',f'from_{time_frame}_peak',f'from_{time_frame}_bottom']].iloc[:1]
            df_name = ['Open','High','Low']
            coef_1 = 4
        elif ta_name == 'fibo':
            df = ta_df[['stock',f'{time_frame}_trend',f'{time_frame}_fibo_382',f'{time_frame}_fibo_500',f'{time_frame}_fibo_618',f'from_{time_frame}_fibo_382',f'from_{time_frame}_fibo_500',f'from_{time_frame}_fibo_618']].iloc[:1]
            df_name = ['Xu hướng cũ', 'Fibo 0.382', 'Fibo 0.500', 'Fibo 0.618']
            coef_1 = 5
            coef_2 = 1
        elif ta_name == 'pivot':
            df = ta_df[['stock',f'{time_frame}_pivot_p',f'from_{time_frame}_pivot_p']].iloc[:1]
            df_name = ['Pivot']
            coef_1 = 2
        elif ta_name == 'ma':
            if time_frame == 'month':
                df = ta_df[['stock','ma5','ma20','from_ma5','from_ma20']].iloc[:1]
                df_name = ['MA5','MA20']
            elif time_frame == 'quarter':
                df = ta_df[['stock','ma60','ma120','from_ma60','from_ma120']].iloc[:1]
                df_name = ['MA60','MA120']
            elif time_frame == 'year':
                df = ta_df[['stock','ma240','ma480','from_ma240','from_ma480']].iloc[:1]
                df_name = ['MA240','MA480']
            coef_1 = 3
        df_value = df.iloc[0,1:coef_1].tolist()
        df_from = [None]*coef_2 + df.iloc[0,coef_1:].tolist()

        if ta_name == 'pivot':
            df_order = 3
        else:
            df_order = [i for i in range(1, len(df_name) + 1)]
            
        df = pd.DataFrame({'stock':df['stock'].item(),'name': df_name,'value': df_value,'from': df_from, 'order': df_order})
        df['id'] = time_frame
        df['ta_name'] = ta_name
        df['value'] = df['value'].apply(lambda x: '{:.2f}'.format(x) if isinstance(x, (int, float)) else x)
        df_list.append(df)
    cancat_df = pd.concat(df_list, axis=0)
    return cancat_df

def concat_ta_df(ta_df,input_type):
    df_candle_raw = calculate_candle_ta_df(ta_df,input_type)
    df_pivot_raw = calculate_pivot_ta_df(ta_df,input_type)
    df_ma_raw = calculate_ma_ta_df(ta_df,input_type)
    df_fibo_raw = calculate_fibo_ta_df(ta_df,input_type)

    df_candle = transform_ta_df(df_candle_raw,'candle')
    df_pivot = transform_ta_df(df_pivot_raw,'pivot')
    df_ma = transform_ta_df(df_ma_raw,'ma')
    df_fibo = transform_ta_df(df_fibo_raw,'fibo')

    concat_ta_df = pd.concat([df_candle,df_fibo,df_pivot,df_ma], axis=0)

    df_rsi = calculate_rsi_ta_df(ta_df)
    ta_trend = df_fibo_raw.iloc[:1]

    ta_trend['month_price_trend'] = ta_trend.apply(lambda x: 
        'Tăng' if x['close']>=x['month_fibo_382'] else
        ('Trung lập' if (x['close']>=x['month_fibo_618']) & (x['close']<x['month_fibo_382']) else
        ('Giảm' if x['close']<x['month_fibo_618'] else None)), axis=1)
    ta_trend['quarter_price_trend'] = ta_trend.apply(lambda x: 
        'Tăng' if x['close']>=x['quarter_fibo_382'] else
        ('Trung lập' if (x['close']>=x['quarter_fibo_618']) & (x['close']<x['quarter_fibo_382']) else
        ('Giảm' if x['close']<x['quarter_fibo_618'] else None)), axis=1)
    ta_trend['year_price_trend'] = ta_trend.apply(lambda x: 
        'Tăng' if x['close']>=x['year_fibo_382'] else
        ('Trung lập' if (x['close']>=x['year_fibo_618']) & (x['close']<x['year_fibo_382']) else
        ('Giảm' if x['close']<x['year_fibo_618'] else None)), axis=1)
    
    ta_trend = pd.DataFrame({'rsi_state': df_rsi['rsi_state'], 
                            'month_price_trend': ta_trend['month_price_trend'], 
                            'quarter_price_trend':ta_trend['quarter_price_trend'], 
                            'year_price_trend':ta_trend['year_price_trend']})

    return concat_ta_df, df_candle_raw, df_pivot_raw, df_ma_raw, df_fibo_raw, ta_trend

### 4. Biến đổi dữ liệu cho Visualization

#### Các bảng dữ liệu độ rộng thị trường

In [23]:
#Độ rộng thị trường
def calculate_market_breadth(name, stock_list, final_score_dict, result_type):
    (up_count, down_count, up_value, down_value, up_volume, down_volume, 
    price_up_count, price_down_count, price_unchange_count, price_down_value, price_up_value, price_unchange_value,
    up_liquid, down_liquid
    ) = (0,) * 14

    for stock, df in final_score_dict.items():
        if stock in stock_list:
            if df['t0_score'].iloc[0].item() >= 0:
                up_count += 1
                up_value += df['close'].iloc[0].item() * df['volume'].iloc[0].item()
                up_volume += df['volume'].iloc[0].item()
            if df['t0_score'].iloc[0].item() < 0:
                down_count += 1
                down_value += df['close'].iloc[0].item() * df['volume'].iloc[0].item()
                down_volume += df['volume'].iloc[0].item()
            if df['price_change'].iloc[0].item() > 0:
                price_up_count += 1
                price_up_value += (df['close'].iloc[0].item() * df['volume'].iloc[0].item())/1000000
            if df['price_change'].iloc[0].item() < 0:
                price_down_count += 1
                price_down_value += (df['close'].iloc[0].item() * df['volume'].iloc[0].item())/1000000
            if df['price_change'].iloc[0].item() == 0:
                price_unchange_count += 1
                price_unchange_value += (df['close'].iloc[0].item() * df['volume'].iloc[0].item())/1000000

            if df['liquid_ratio'].iloc[0].item() >= 1:
                up_liquid += 1
            if df['liquid_ratio'].iloc[0].item() < 1:
                down_liquid += 1


    if result_type == 'count': return [name, up_count, down_count]
    if result_type == 'value': return [name, up_value, down_value]
    if result_type == 'volume': return [name, up_volume, down_volume]
    if result_type == 'price_count': return [name, price_up_count, price_down_count, price_unchange_count]
    if result_type == 'price_value': return [name, price_up_value, price_down_value, price_unchange_value]
    if result_type == 'liquid_ratio': return [name, up_liquid, down_liquid]

#Tạo bảng độ rộng theo dòng tiền
perform_A = calculate_market_breadth('A', stock_classification_filtered[stock_classification_filtered['industry_perform']=='A']['stock'].tolist(), final_score_dict, 'count')
perform_B = calculate_market_breadth('B', stock_classification_filtered[stock_classification_filtered['industry_perform']=='B']['stock'].tolist(), final_score_dict, 'count')
perform_C = calculate_market_breadth('C', stock_classification_filtered[stock_classification_filtered['industry_perform']=='C']['stock'].tolist(), final_score_dict, 'count')
perform_D = calculate_market_breadth('D', stock_classification_filtered[stock_classification_filtered['industry_perform']=='D']['stock'].tolist(), final_score_dict, 'count')
cap_large = calculate_market_breadth('cap_large', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='large']['stock'].tolist(), final_score_dict, 'count')
cap_mid = calculate_market_breadth('cap_mid', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='mid']['stock'].tolist(), final_score_dict, 'count')
cap_small = calculate_market_breadth('cap_small', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='small']['stock'].tolist(), final_score_dict, 'count')
cap_penny = calculate_market_breadth('cap_penny', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='penny']['stock'].tolist(), final_score_dict, 'count')
all_stock_count = calculate_market_breadth('all_stock_count', stock_classification_filtered['stock'].tolist(), final_score_dict, 'count')
all_stock_value = calculate_market_breadth('all_stock_value', stock_classification_filtered['stock'].tolist(), final_score_dict, 'value')
all_stock_volume = calculate_market_breadth('all_stock_volume', stock_classification_filtered['stock'].tolist(), final_score_dict, 'volume')

temp_list = [perform_A ,perform_B, perform_C, perform_D, cap_large, cap_mid, cap_small, cap_penny, all_stock_count, all_stock_value, all_stock_volume]
temp_market_breadth_flow_df = pd.DataFrame(temp_list, columns=['name','Tiền vào','Tiền ra'])
market_breadth_flow_df =temp_market_breadth_flow_df.set_index('name').transpose().reset_index().rename(columns={"index":"name"})

#Tạo bảng độ rộng theo tăng giảm giá
all_stock_price_count = calculate_market_breadth('all_stock_price_count', stock_classification_filtered['stock'].tolist(), final_score_dict, 'price_count')
all_stock_price_value = calculate_market_breadth('all_stock_price_value', stock_classification_filtered['stock'].tolist(), final_score_dict, 'price_value')

market_breadth_price_df = pd.DataFrame([all_stock_price_count,all_stock_price_value], columns=['name','Tăng giá','Giảm giá','Không đổi']).set_index('name').transpose().reset_index().rename(columns={"index":"name"})

#Tạo bảng độ rộng theo thanh khoản
perform_A = calculate_market_breadth('A', stock_classification_filtered[stock_classification_filtered['industry_perform']=='A']['stock'].tolist(), final_score_dict, 'liquid_ratio')
perform_B = calculate_market_breadth('B', stock_classification_filtered[stock_classification_filtered['industry_perform']=='B']['stock'].tolist(), final_score_dict, 'liquid_ratio')
perform_C = calculate_market_breadth('C', stock_classification_filtered[stock_classification_filtered['industry_perform']=='C']['stock'].tolist(), final_score_dict, 'liquid_ratio')
perform_D = calculate_market_breadth('D', stock_classification_filtered[stock_classification_filtered['industry_perform']=='D']['stock'].tolist(), final_score_dict, 'liquid_ratio')
cap_large = calculate_market_breadth('cap_large', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='large']['stock'].tolist(), final_score_dict, 'liquid_ratio')
cap_mid = calculate_market_breadth('cap_mid', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='mid']['stock'].tolist(), final_score_dict, 'liquid_ratio')
cap_small = calculate_market_breadth('cap_small', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='small']['stock'].tolist(), final_score_dict, 'liquid_ratio')
cap_penny = calculate_market_breadth('cap_penny', stock_classification_filtered[stock_classification_filtered['marketcap_group']=='penny']['stock'].tolist(), final_score_dict, 'liquid_ratio')
all_stock_count = calculate_market_breadth('all_stock_count', stock_classification_filtered['stock'].tolist(), final_score_dict, 'liquid_ratio')

temp_list = [perform_A ,perform_B, perform_C, perform_D, cap_large, cap_mid, cap_small, cap_penny, all_stock]
market_breadth_liquid_df = pd.DataFrame(temp_list, columns=['name','> 100%','< 100%']).set_index('name').transpose().reset_index().rename(columns={"index":"name"})

In [24]:
#Tính độ rộng phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = date_series.copy().iloc[:100]
for stock, df in stock_score_dict_filtered.items():
    temp_df[stock] = stock_score_dict_filtered[stock]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

market_breath_df = date_series.copy().iloc[:100]

industry_name_breadth_dict = {}
for key in industry_name.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_name']==key]['stock'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    market_breath_df[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in industry_perform.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_perform']==key]['stock'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    market_breath_df[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in marketcap_group.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['marketcap_group']==key]['stock'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    market_breath_df[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in all_stock.keys():
    stock_list = stock_classification_filtered['stock'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    market_breath_df[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

In [25]:
def adjusted_score(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

for stock, df in stock_score_dict_filtered.items():
    industry_name_name = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    industry_perform_name = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    marketcap_group_name = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    df[f't0_industry_name'] = adjusted_score(df['t0_score'], market_breath_df[industry_name_name])
    df[f't0_industry_perform'] = adjusted_score(df['t0_score'], market_breath_df[industry_perform_name])
    df[f't0_marketcap_group'] = adjusted_score(df['t0_score'], market_breath_df[marketcap_group_name])
    df[f't0_all_stock'] = adjusted_score(df['t0_score'], market_breath_df['all_stock'])

#### Các bảng dữ liệu dòng tiền tổng hợp cho các nhóm cổ phiếu

In [26]:
#Điều chỉnh loại bỏ các cổ phiếu gây đột biến
def fix_score(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score
    
def add_group_score(group_stock, group_name, stock_score_dict_filtered=stock_score_dict_filtered):
    for key in group_stock.keys():

        score_df = date_series.copy()
        if group_name == 'all_stock':
            temp_stock_list = stock_classification_filtered['stock']
        else:
            temp_stock_list = stock_classification_filtered[stock_classification_filtered[f'{group_name}']==key]['stock']

        for stock in temp_stock_list:
            score_df[stock] = stock_score_dict_filtered[stock][f't0_{group_name}']

        max_percent = max(0.1, min(5*(1/len(temp_stock_list)), 0.6))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for stock in temp_stock_list:
                    score_df[stock] = score_df.iloc[:, 1:].apply(fix_score, axis=1, args=(stock, max_percent, mark))
            if mark[0] == 0:
                break

        for stock in temp_stock_list:
            stock_score_dict_filtered[stock][f't0_{group_name}'] = score_df[stock]

add_group_score(industry_name, 'industry_name')
add_group_score(industry_perform, 'industry_perform')
add_group_score(marketcap_group, 'marketcap_group')
add_group_score(all_stock, 'all_stock')

In [27]:
#Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
all_stock_score_df = date_series.copy()
for nganh in all_stock.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered['stock']:
        score_df[stock] = stock_score_dict_filtered[stock]['t0_all_stock']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    all_stock_score_df[nganh] = score_df['total']
all_stock_score_df['week'] = all_stock_score_df['date'].dt.strftime('%U-%Y')
all_stock_score_df['month'] = all_stock_score_df['date'].dt.strftime('%m-%Y')
all_stock_score_df['week_day'] = all_stock_score_df['date'].dt.day_name()
all_stock_score_df['day_num'] = all_stock_score_df['date'].dt.day

industry_score_df = date_series.copy()
for nganh in industry_name.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock']:
        score_df[stock] = stock_score_dict_filtered[stock]['t0_industry_name']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    industry_score_df[nganh] = score_df['total']
industry_score_df['week'] = industry_score_df['date'].dt.strftime('%U-%Y')
industry_score_df['month'] = industry_score_df['date'].dt.strftime('%m-%Y')
industry_score_df['week_day'] = industry_score_df['date'].dt.day_name()
industry_score_df['day_num'] = industry_score_df['date'].dt.day

industry_perform_score_df = date_series.copy()
for group in industry_perform.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_perform']==group]['stock']:
        score_df[stock] = stock_score_dict_filtered[stock]['t0_industry_perform']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    industry_perform_score_df[group] = score_df['total']
industry_perform_score_df['week'] = industry_score_df['date'].dt.strftime('%U-%Y')
industry_perform_score_df['month'] = industry_score_df['date'].dt.strftime('%m-%Y')
industry_perform_score_df['week_day'] = industry_perform_score_df['date'].dt.day_name()
industry_perform_score_df['day_num'] = industry_perform_score_df['date'].dt.day

marketcap_group_score_df = date_series.copy()
for marketcap in marketcap_group.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==marketcap]['stock']:
        score_df[stock] = stock_score_dict_filtered[stock]['t0_marketcap_group']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    marketcap_group_score_df[marketcap] = score_df['total']
marketcap_group_score_df['week'] = industry_score_df['date'].dt.strftime('%U-%Y')
marketcap_group_score_df['month'] = industry_score_df['date'].dt.strftime('%m-%Y')
marketcap_group_score_df['week_day'] = marketcap_group_score_df['date'].dt.day_name()
marketcap_group_score_df['day_num'] = marketcap_group_score_df['date'].dt.day

#### Các bảng dữ liệu thông tin nhanh về các chỉ số thị trường

In [28]:
#Tạo bảng chỉ số thị trường
temp_dict = {}
index_dict_clone = copy.deepcopy(index_dict)
for index, df in index_dict_clone.items():
    df['basic'] = df['close'].shift(-1)
    df['change_percent'] = (df['close'] - df['basic'])/df['basic']
    df['change_value'] = (df['close'] - df['basic'])
    df['range'] = (df['high'] - df['low'])
    df['volume'] = df['volume']/1000
    df['value'] = df['value']/1000000000
    temp_dict[index] = df[['close','change_percent','change_value','range','volume','value']].iloc[0].tolist()

mareket_index_df = pd.DataFrame.from_dict(temp_dict, orient='index').reset_index().sort_index(ascending=False)
mareket_index_df.columns = ['index','close','change_percent','change_value','range','volume','value']
mareket_index_df['order'] = mareket_index_df['index'].apply(lambda x: 1 if x == 'VNINDEX' else
                                                                     (2 if x == 'VN30' else
                                                                     (3 if x == 'HNXINDEX' else 4)))

In [29]:
# #Tạo bảng chỉ số phái sinh
# temp_dict = {}
# index_dict_fu_clone = copy.deepcopy(index_fu_dict)
# for index, df in index_dict_fu_clone.items():
#     df['basic'] = df['close'].shift(-1)
#     df['change_percent'] = (df['close'] - df['basic'])/df['basic']
#     df['change_value'] = (df['close'] - df['basic'])
#     df['differrence'] = df['close'] - index_dict['VN30']['close'].iloc[0].item()
#     temp_dict[index] = df[['close','change_percent','change_value','differrence','volume','open_volume']].iloc[0].tolist()

# fu_index_df = pd.DataFrame.from_dict(temp_dict, orient='index').reset_index().sort_values('index', ascending=False)
# fu_index_df.columns = ['index','close','change_percent','change_value','differrence','volume','open_volume']

# temp_dict = {}
# for index, df in index_fu_nn_dict.items():
#     temp_df = pd.DataFrame()
#     temp_df['index'] = pd.Series(index[:-3])
#     temp_df['nn_buy'] = pd.Series(df.iloc[0]['buy_volume'])
#     temp_df['nn_sell'] = pd.Series(df.iloc[0]['sell_volume'])
#     temp_dict[index] = temp_df
# temp_df = pd.concat(temp_dict.values(), axis=0)
# fu_index_df = pd.merge(fu_index_df, temp_df, on='index')

In [30]:
#Tạo dữ liệu mua bán phiên hiện tại khối ngoại và tự doanh
def calculate_nn_td_buy_sell(index_name):
    temp_dict_nn = {}
    temp_dict_nn['KLGD_NN'] = index_td_nn_dict[f'{index_name}_NN'].iloc[0][['buy_volume','sell_volume','net_volume']].tolist()
    temp_dict_nn['GTGD_NN'] = index_td_nn_dict[f'{index_name}_NN'].iloc[0][['buy_value','sell_value','net_value']].tolist()
    nn_buy_sell_df = pd.DataFrame.from_dict(temp_dict_nn, orient='index').reset_index()
    nn_buy_sell_df.columns = ['type','Mua','Bán','Mua-Bán']
    nn_buy_sell_df = nn_buy_sell_df.set_index('type').transpose()

    temp_dict_td = {}
    temp_dict_td['KLGD_TD'] = index_td_nn_dict[f'{index_name}_TD'].iloc[0][['buy_volume','sell_volume','net_volume']].tolist()
    temp_dict_td['GTGD_TD'] = index_td_nn_dict[f'{index_name}_TD'].iloc[0][['buy_value','sell_value','net_value']].tolist()
    td_buy_sell_df = pd.DataFrame.from_dict(temp_dict_td, orient='index').reset_index()
    td_buy_sell_df.columns = ['type','Mua','Bán','Mua-Bán']
    td_buy_sell_df = td_buy_sell_df.set_index('type').transpose()

    nn_td_buy_sell_df = pd.concat([nn_buy_sell_df,td_buy_sell_df],axis=1)
    nn_td_buy_sell_df['order'] = [1,2,3]

    return nn_td_buy_sell_df

nn_td_buy_sell_hsx = calculate_nn_td_buy_sell('VNINDEX')
nn_td_buy_sell_hsx['id'] = 'HSX'
nn_td_buy_sell_hsx['order_id'] = 1
nn_td_buy_sell_hnx = calculate_nn_td_buy_sell('HNXINDEX')
nn_td_buy_sell_hnx['id'] = 'HNX'
nn_td_buy_sell_hnx['order_id'] = 2
nn_td_buy_sell_upcom = calculate_nn_td_buy_sell('UPINDEX')
nn_td_buy_sell_upcom['id'] = 'UPCOM'
nn_td_buy_sell_upcom['order_id'] = 3

nn_td_buy_sell_df = pd.concat([nn_td_buy_sell_hsx,nn_td_buy_sell_hnx,nn_td_buy_sell_upcom],axis=0).reset_index().rename(columns={'index':'type'})

#Tạo dữ liệu lịch sử 20p khối ngoại và tự doanh
nn_20p_df_hsx = index_td_nn_dict['VNINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_hsx = index_td_nn_dict['VNINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_hsx = nn_20p_df_hsx.merge(td_20p_df_hsx, how='left', on='date')
nn_td_20p_df_hsx['id'] = 'HSX'

nn_20p_df_hnx = index_td_nn_dict['HNXINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_hnx = index_td_nn_dict['HNXINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_hnx = nn_20p_df_hnx.merge(td_20p_df_hnx, how='left', on='date')
nn_td_20p_df_hnx['id'] = 'HNX'

nn_20p_df_upcom = index_td_nn_dict['UPINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_upcom = index_td_nn_dict['UPINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_upcom = nn_20p_df_upcom.merge(td_20p_df_upcom, how='left', on='date')
nn_td_20p_df_upcom['id'] = 'UPCOM'

nn_td_20p_df = pd.concat([nn_td_20p_df_hsx,nn_td_20p_df_hnx,nn_td_20p_df_upcom],axis=0)

In [31]:
def create_nn_td_top_stock(stock_dict):
    today = date_series['date'][0]
    yesterday = date_series['date'][1]
    the_day_before = date_series['date'][2]

    #Tạo ra top cổ phiếu mua bán của NN
    top_stock_dict = {}
    for stock, df in stock_dict.items():
        if not df.empty:
            if df['date'][0] == today:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
            elif df['date'][0] == yesterday:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
            elif df['date'][0] == the_day_before:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
    top_stock_df = pd.DataFrame.from_dict(top_stock_dict, orient='index').reset_index()
    top_stock_df.columns = df.columns
    top_stock_df['net_values'] = (top_stock_df['buy_value'] - top_stock_df['sell_value'])/1000000000
    top_stock_df['stock'] = top_stock_df['stock'].apply(lambda x: x[:3])

    top_sell = top_stock_df[top_stock_df['net_values']<0].sort_values('net_values')[['stock','date','net_values']].rename(columns={'stock':'sell_stock','net_values':'sell_value'}).reset_index(drop=True).head(20)
    top_buy = top_stock_df[top_stock_df['net_values']>0].sort_values('net_values', ascending=False)[['stock','net_values']].rename(columns={'stock':'buy_stock','net_values':'buy_value'}).reset_index(drop=True).head(20)
    top_stock_df = pd.concat([top_sell,top_buy], axis=1)

    return top_stock_df

try:
    nn_top_stock_hsx = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HSX']['stock'].tolist()})
    nn_top_stock_hsx.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_hnx = pd.DataFrame(columns=['nn_sell_stock', 'nn_date', 'nn_sell_value', 'nn_buy_stock', 'nn_buy_value'])
try:
    td_top_stock_hsx = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HSX']['stock'].tolist()})
    td_top_stock_hsx.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_hsx = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_hsx = pd.concat([nn_top_stock_hsx,td_top_stock_hsx],axis=1)
nn_td_top_stock_hsx['id'] = 'HSX'

try:
    nn_top_stock_hnx = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HNX']['stock'].tolist()})
    nn_top_stock_hnx.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_hnx = pd.DataFrame(columns=['nn_sell_stock', 'nn_date', 'nn_sell_value', 'nn_buy_stock', 'nn_buy_value'])

try:
    td_top_stock_hnx = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HNX']['stock'].tolist()})
    td_top_stock_hnx.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_hnx = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_hnx = pd.concat([nn_top_stock_hnx,td_top_stock_hnx],axis=1)
nn_td_top_stock_hnx['id'] = 'HNX'

try:
    nn_top_stock_upcom = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='UPCOM']['stock'].tolist()})
    nn_top_stock_upcom.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_upcom = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

try:
    td_top_stock_upcom = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='UPCOM']['stock'].tolist()})
    td_top_stock_upcom.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_upcom = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_upcom = pd.concat([nn_top_stock_upcom,td_top_stock_upcom],axis=1)
nn_td_top_stock_upcom['id'] = 'UPCOM'

nn_td_top_stock = pd.concat([nn_td_top_stock_hsx,nn_td_top_stock_hnx,nn_td_top_stock_upcom],axis=0)

#### Các bảng dữ liệu dòng tiền trong phiên

In [32]:
#Dòng tiền vào các nhóm cổ phiếu lớn
temp_df1 = industry_perform_score_df.iloc[:1, 1:-4]
temp_df1 = temp_df1.transpose().reset_index().rename(columns={'index':'industry_perform',0:'industry_perform_value'})
temp_df2 = marketcap_group_score_df.iloc[:1, 1:-4]
temp_df2 = temp_df2.transpose().reset_index().rename(columns={'index':'marketcap_group',0:'marketcap_group_value'})
stock_group_score_t0 = pd.concat([temp_df1, temp_df2], axis=1)
stock_group_score_t0['industry_perform'] = stock_group_score_t0['industry_perform'].map(name_map_dict)
stock_group_score_t0['marketcap_group'] = stock_group_score_t0['marketcap_group'].map(name_map_dict)

#Dòng tiền vào các ngành
perform_A_list = stock_classification[stock_classification['industry_perform']=='A']['industry_name'].unique()
perform_B_list = stock_classification[stock_classification['industry_perform']=='B']['industry_name'].unique()
perform_C_list = stock_classification[stock_classification['industry_perform']=='C']['industry_name'].unique()
perform_D_list = stock_classification[stock_classification['industry_perform']=='D']['industry_name'].unique()
industry_score_t0 = industry_score_df.iloc[:1, 1:-4]
industry_score_t0 = industry_score_t0.transpose().reset_index().rename(columns={'index':'industry',0:'value'})
perform_A = industry_score_t0[industry_score_t0['industry'].isin(perform_A_list)].reset_index(drop=True).rename(columns={'industry':'industry_1','value':'value_1'})
perform_B = industry_score_t0[industry_score_t0['industry'].isin(perform_B_list)].reset_index(drop=True).rename(columns={'industry':'industry_2','value':'value_2'})
perform_C = industry_score_t0[industry_score_t0['industry'].isin(perform_C_list)].reset_index(drop=True).rename(columns={'industry':'industry_3','value':'value_3'})
perform_D = industry_score_t0[industry_score_t0['industry'].isin(perform_D_list)].reset_index(drop=True).rename(columns={'industry':'industry_4','value':'value_4'})
industry_score_t0 = pd.concat([perform_A,perform_B,perform_C,perform_D],axis=1)
industry_score_t0['industry_1'] = industry_score_t0['industry_1'].map(name_map_dict)
industry_score_t0['industry_2'] = industry_score_t0['industry_2'].map(name_map_dict)
industry_score_t0['industry_3'] = industry_score_t0['industry_3'].map(name_map_dict)
industry_score_t0['industry_4'] = industry_score_t0['industry_4'].map(name_map_dict)

#### Bảng tâm lý thị trường và hệ số thanh khoản

In [33]:
market_sentiment_A = date_series.copy()
market_sentiment_B = date_series.copy()
market_sentiment_C = date_series.copy()
market_sentiment_D = date_series.copy()
for stock, df in stock_score_dict_filtered.items():
    if stock in stock_classification[stock_classification['industry_perform']=='A']['stock'].tolist():
        market_sentiment_A[stock] = df['t0_score']
    elif stock in stock_classification[stock_classification['industry_perform']=='B']['stock'].tolist():
        market_sentiment_B[stock] = df['t0_score']
    elif stock in stock_classification[stock_classification['industry_perform']=='C']['stock'].tolist():
        market_sentiment_C[stock] = df['t0_score']
    elif stock in stock_classification[stock_classification['industry_perform']=='D']['stock'].tolist():
        market_sentiment_D[stock] = df['t0_score']

market_sentiment = date_series.copy()
market_sentiment['A>0'] = market_sentiment_A.iloc[:,1:].apply(lambda x: (x > 0).sum(), axis=1)
market_sentiment['A'] = market_sentiment_A.iloc[:,1:].count(axis=1)
market_sentiment['B>0'] = market_sentiment_B.iloc[:,1:].apply(lambda x: (x > 0).sum(), axis=1)
market_sentiment['B'] = market_sentiment_B.iloc[:,1:].count(axis=1)
market_sentiment['C>0'] = market_sentiment_C.iloc[:,1:].apply(lambda x: (x > 0).sum(), axis=1)
market_sentiment['C'] = market_sentiment_C.iloc[:,1:].count(axis=1)
market_sentiment['D>0'] = market_sentiment_D.iloc[:,1:].apply(lambda x: (x > 0).sum(), axis=1)
market_sentiment['D'] = market_sentiment_D.iloc[:,1:].count(axis=1)
market_sentiment['total'] = market_sentiment['A']*0.618+market_sentiment['B']*0.5+market_sentiment['C']*0.5+market_sentiment['D']*0.382
market_sentiment['total>0'] = market_sentiment['A>0']*0.618+market_sentiment['B>0']*0.5+market_sentiment['C>0']*0.5+market_sentiment['D>0']*0.382
market_sentiment['state_ratio'] = market_sentiment['total>0']/market_sentiment['total']

market_sentiment = market_sentiment[['date','state_ratio']]
market_sentiment['market_state'] = market_sentiment['state_ratio'].apply(lambda x: 'Bán mạnh' if x < 0.146 else
                                                                            ('Bán' if (x >= 0.146) & (x < 0.382) else
                                                                            ('Trung lập' if (x >= 0.382) & (x < 0.618) else
                                                                            ('Mua' if (x >= 0.618) & (x < 0.85) else 'Mua mạnh'))))

market_sentiment['sentiment_ratio'] = market_sentiment['state_ratio'][::-1].rolling(window=3).mean()[::-1]
market_sentiment['sentiment_state'] = market_sentiment['sentiment_ratio'].apply(lambda x: 'Sợ hãi' if x < 0.146 else
                                                                            ('Tiêu cực' if (x >= 0.146) & (x < 0.382) else
                                                                            ('Lưỡng lự' if (x >= 0.382) & (x < 0.618) else
                                                                            ('Tích cực' if (x >= 0.618) & (x < 0.85) else 'Hưng phấn'))))
market_sentiment['sentiment_ratio'] = market_sentiment['sentiment_ratio']*100
market_sentiment = market_sentiment.head(60)

In [34]:
#Tạo hệ số thanh khoản
t0_value = 0
t5_value = 0
for stock, df in stock_dict.items():
    t0_value += df.iloc[0]['volume'].item()
    t5_value += df.iloc[0]['ma5_V'].item()
t5_value = t5_value*time_percent
liquidity_coef_df = pd.DataFrame({'t0_value':[t0_value],'t5_value':[t5_value],'liquidity_coef':[(t0_value/t5_value)]}, index=[0])
liquidity_coef_df['min'] = 0
liquidity_coef_df['mid1'] = 0.3
liquidity_coef_df['mid2'] = 0.6
liquidity_coef_df['mid3'] = 0.9
liquidity_coef_df['mid4'] = 1.2
liquidity_coef_df['max'] = 1.5

#Tạo hệ số trạng thái thị trường
market_state_coef = market_sentiment[['state_ratio','market_state']].head(1)
market_state_coef['min'] = 0
market_state_coef['mid1'] = 0.146
market_state_coef['mid2'] = 0.382
market_state_coef['mid3'] = 0.618
market_state_coef['mid4'] = 0.85
market_state_coef['max'] = 1

In [57]:
market_state_coef

,state_ratio,market_state,min,mid1,mid2,mid3,mid4,max
0,0.852403,Mua mạnh,0,0.146,0.382,0.618,0.85,1


In [35]:
#Tạo bảng hệ số thanh khoản cho ngành
industry_name_liquid_dict = {}
for group in stock_classification['industry_name'].unique():
    t0_value = 0
    t5_value = 0
    for stock in stock_classification[stock_classification['industry_name']==group]['stock']:
        df = stock_dict[stock]
        t0_value += df.iloc[0]['volume'].item()
        t5_value += df.iloc[0]['ma5_V'].item()
    t5_value = t5_value*time_percent
    industry_name_liquid_dict[group] = t0_value/t5_value
industry_name_liquid_df = pd.DataFrame.from_dict(industry_name_liquid_dict, orient='index').reset_index().rename(columns={'index':'industry_name', 0:'liquid_ratio'})

perform_A_list = stock_classification[stock_classification['industry_perform']=='A']['industry_name'].unique()
perform_B_list = stock_classification[stock_classification['industry_perform']=='B']['industry_name'].unique()
perform_C_list = stock_classification[stock_classification['industry_perform']=='C']['industry_name'].unique()
perform_D_list = stock_classification[stock_classification['industry_perform']=='D']['industry_name'].unique()

perform_A = industry_name_liquid_df[industry_name_liquid_df['industry_name'].isin(perform_A_list)].reset_index(drop=True).rename(columns={'industry_name':'industry_1','liquid_ratio':'liquid_ratio_1'})
perform_B = industry_name_liquid_df[industry_name_liquid_df['industry_name'].isin(perform_B_list)].reset_index(drop=True).rename(columns={'industry_name':'industry_2','liquid_ratio':'liquid_ratio_2'})
perform_C = industry_name_liquid_df[industry_name_liquid_df['industry_name'].isin(perform_C_list)].reset_index(drop=True).rename(columns={'industry_name':'industry_3','liquid_ratio':'liquid_ratio_3'})
perform_D = industry_name_liquid_df[industry_name_liquid_df['industry_name'].isin(perform_D_list)].reset_index(drop=True).rename(columns={'industry_name':'industry_4','liquid_ratio':'liquid_ratio_4'})

industry_name_liquid_df = pd.concat([perform_A,perform_B,perform_C,perform_D],axis=1)
industry_name_liquid_df['industry_1'] = industry_name_liquid_df['industry_1'].map(name_map_dict)
industry_name_liquid_df['industry_2'] = industry_name_liquid_df['industry_2'].map(name_map_dict)
industry_name_liquid_df['industry_3'] = industry_name_liquid_df['industry_3'].map(name_map_dict)
industry_name_liquid_df['industry_4'] = industry_name_liquid_df['industry_4'].map(name_map_dict)

In [36]:
#Tạo bảng hệ số thanh khoản cho nhóm hiệu suất
industry_perform_liquid_dict = {}
for group in stock_classification['industry_perform'].unique():
    t0_value = 0
    t5_value = 0
    for stock in stock_classification[stock_classification['industry_perform']==group]['stock']:
        df = stock_dict[stock]
        t0_value += df.iloc[0]['volume'].item()
        t5_value += df.iloc[0]['ma5_V'].item()
    t5_value = t5_value*time_percent
    industry_perform_liquid_dict[group] = t0_value/t5_value
industry_perform_liquid_df = pd.DataFrame.from_dict(industry_perform_liquid_dict, orient='index').reset_index().rename(columns={'index':'industry_perform', 0:'liquid_ratio'})
industry_perform_liquid_df['industry_perform'] = industry_perform_liquid_df['industry_perform'].map(name_map_dict)

In [37]:
#Tạo bảng hệ số thanh khoản cho nhóm vốn hoá
marketcap_group_liquid_dict = {}
for group in stock_classification['marketcap_group'].unique():
    t0_value = 0
    t5_value = 0
    for stock in stock_classification[stock_classification['marketcap_group']==group]['stock']:
        df = stock_dict[stock]
        t0_value += df.iloc[0]['volume'].item()
        t5_value += df.iloc[0]['ma5_V'].item()
    t5_value = t5_value*time_percent
    marketcap_group_liquid_dict[group] = t0_value/t5_value
marketcap_group_liquid_df = pd.DataFrame.from_dict(marketcap_group_liquid_dict, orient='index').reset_index().rename(columns={'index':'marketcap_group', 0:'liquid_ratio'})
marketcap_group_liquid_df['cap_order'] = marketcap_group_liquid_df['marketcap_group'].apply(lambda x: 0 if x=='large' else (
                                                                                          1 if x=='mid' else(
                                                                                          2 if x=='small' else 3)))
marketcap_group_liquid_df['marketcap_group'] = marketcap_group_liquid_df['marketcap_group'].map(name_map_dict)

#### Các bảng dữ liệu cho dòng tiền trong tuần và dòng tiền trong tháng của các nhóm cổ phiếu

In [38]:
def fill_month_flow(series):
    new_series = series.copy()
    for i in range(len(series) - 1):
        if i == 0:
            fill_value = 0
            new_series[i] = fill_value
        else:
            fill_value = new_series[i-1]
            if pd.isna(series[i]):
                if not series[i:-1].isna().all():
                    new_series[i] = fill_value
    return new_series

In [39]:
#Tạo dữ liệu cho toàn bộ cổ phiếu theo dõi
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = all_stock_score_df[all_stock_score_df['week'] == all_stock_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars='all_stock', var_name='all_stock', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'all_stock'], columns='id', values='value', aggfunc='first').reset_index()

all_stock_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','all_stock'], how='outer')

all_stock_score_week['day_index'] = all_stock_score_week['week_day'].map(week_day_index)
all_stock_score_week = all_stock_score_week.sort_values('day_index')

#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = all_stock_score_df[all_stock_score_df['month'] == all_stock_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'

    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars='all_stock', var_name='all_stock', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'all_stock'], columns='id', values='value', aggfunc='first').reset_index()

all_stock_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','all_stock'], how='outer')

In [40]:
#Tạo dữ liệu cho các nhóm vốn hoá
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = marketcap_group_score_df[marketcap_group_score_df['week'] == marketcap_group_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars=['large', 'mid', 'small', 'penny'], var_name='marketcap', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'marketcap'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tuần
marketcap_group_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','marketcap'], how='outer')
marketcap_group_score_week['day_index'] = marketcap_group_score_week['week_day'].map(week_day_index)
marketcap_group_score_week['marketcap'] = marketcap_group_score_week['marketcap'].map(name_map_dict)
marketcap_group_score_week = marketcap_group_score_week.sort_values('day_index')

#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = marketcap_group_score_df[marketcap_group_score_df['month'] == marketcap_group_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'
    
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars=['large', 'mid', 'small', 'penny'], var_name='marketcap', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'marketcap'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tháng
marketcap_group_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','marketcap'], how='outer')
marketcap_group_score_month['marketcap'] = marketcap_group_score_month['marketcap'].map(name_map_dict)


In [41]:
#Tạo dữ liệu cho các nhóm hiệu suất
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = industry_perform_score_df[industry_perform_score_df['week'] == industry_perform_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars=['A', 'B', 'C', 'D'], var_name='performance', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'performance'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tuần
industry_perform_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','performance'], how='outer')
industry_perform_score_week['day_index'] = industry_perform_score_week['week_day'].map(week_day_index)
industry_perform_score_week['performance'] = industry_perform_score_week['performance'].map(name_map_dict)
industry_perform_score_week = industry_perform_score_week.sort_values('day_index')

#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = industry_perform_score_df[industry_perform_score_df['month'] == industry_perform_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'
    
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars=['A', 'B', 'C', 'D'], var_name='performance', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'performance'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh các 2 tháng
industry_perform_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','performance'], how='outer')
industry_perform_score_month['performance'] = industry_perform_score_month['performance'].map(name_map_dict)

In [42]:
#Tạo dữ liệu cho các nhóm ngành nghề
industry_list = industry_score_df.columns[1:-4].tolist()
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = industry_score_df[industry_score_df['week'] == industry_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars=industry_list, var_name='industry', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'industry'], columns='id', values='value', aggfunc='first').reset_index()

industry_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','industry'], how='outer')

industry_score_week['day_index'] = industry_score_week['week_day'].map(week_day_index)
industry_score_week = industry_score_week.sort_values('day_index')
industry_score_week['industry'] = industry_score_week['industry'].map(name_map_dict)

#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = industry_score_df[industry_score_df['month'] == industry_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'
    
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars=industry_list, var_name='industry', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'industry'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tháng
industry_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','industry'], how='outer')
industry_score_month['industry'] = industry_score_month['industry'].map(name_map_dict)

#### Các bảng dữ liệu chỉ số kĩ thuật của các nhóm cổ phiếu

In [43]:
#Tính toán index cho biểu đồ nến của các nhóm cổ phiếu
def calculate_index(stock_group, name):
    index_price_df = date_series.copy()
    index_price_dict = {'open':None, 'high':None, 'low':None, 'close':None, 'volume':None}
    for key in index_price_dict.keys():
        index_price_dict[key] = date_series.copy()

        for stock, df in stock_group[name].items():
            if key == 'open': index_price_dict[key][stock] = (df['open'] - df['close'].shift(-1))/df['close'].shift(-1)
            elif key == 'high': index_price_dict[key][stock] = (df['high'] - df['close'].shift(-1))/df['close'].shift(-1)
            elif key == 'low': index_price_dict[key][stock] = (df['low'] - df['close'].shift(-1))/df['close'].shift(-1)
            elif key == 'close': index_price_dict[key][stock] = (df['close'] - df['close'].shift(-1))/df['close'].shift(-1)
            elif key == 'volume': index_price_dict[key][stock] = df['volume']
        
        if key != 'volume':
            index_price_dict[key]['mean'] = index_price_dict[key].iloc[:,1:].mean(axis=1)
            index_price_dict[key] = index_price_dict[key].fillna(0)
            index_price_df[f'{key}_change'] = index_price_dict[key]['mean']
        else:
            index_price_dict[key]['sum'] = index_price_dict[key].iloc[:,1:].sum(axis=1)
            index_price_df[f'volume'] = index_price_dict[key]['sum']


    index_price_df['close'] = (index_price_df['close_change']*1000)[::-1].cumsum()[::-1]
    index_price_df['high'] = index_price_df['high_change']*1000/2 + index_price_df['close'].shift(-1)
    index_price_df['low'] = index_price_df['low_change']*1000/2 + index_price_df['close'].shift(-1)
    index_price_df['open'] = index_price_df['open_change']*1000 + index_price_df['close'].shift(-1)
    index_price_df = index_price_df[['date','open','high','low','close','volume']].fillna(0)

    return index_price_df

In [44]:
#Tính toán index price cho T2MINDEX
all_stock_index_dict = {}
all_stock_index_price = pd.DataFrame()
for group, df in all_stock.items():
    all_stock_index_dict[group] = calculate_index(all_stock, group)
    all_stock_index_dict[group]['stock'] = group
    all_stock_index_dict[group]['ma5_V'] = all_stock_index_dict[group]['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    all_stock_index_dict[group]['liquid_ratio'] = all_stock_index_dict[group]['volume'] / (all_stock_index_dict[group]['ma5_V'])
    all_stock_index_dict[group]['liquid_ratio'].iloc[0] = all_stock_index_dict[group]['volume'].iloc[0] / ((all_stock_index_dict[group]['ma5_V']).iloc[0]*time_percent)
for index_name, df in all_stock_index_dict.items():
    all_stock_index_price = pd.concat([all_stock_index_price, df.iloc[:60]], axis=0)
all_stock_index_price['color'] = all_stock_index_price.apply(lambda x: 'red' if x['close']<x['open'] else 'green', axis=1)

#Tính toán index price cho các index ngành
industry_index_dict = {}
all_industry_index_price = pd.DataFrame()
for group, df in industry_name.items():
    industry_index_dict[group] = calculate_index(industry_name, group)
    industry_index_dict[group]['stock'] = group
    industry_index_dict[group]['ma5_V'] = industry_index_dict[group]['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    industry_index_dict[group]['liquid_ratio'] = industry_index_dict[group]['volume'] / (industry_index_dict[group]['ma5_V'])
    industry_index_dict[group]['liquid_ratio'].iloc[0] = industry_index_dict[group]['volume'].iloc[0] / ((industry_index_dict[group]['ma5_V']).iloc[0]*time_percent)
for index_name, df in industry_index_dict.items():
    all_industry_index_price = pd.concat([all_industry_index_price, df.iloc[:60]], axis=0)
all_industry_index_price['stock'] = all_industry_index_price['stock'].map(name_map_dict)
all_industry_index_price['color'] = all_industry_index_price.apply(lambda x: 'red' if x['close']<x['open'] else 'green', axis=1)

#Tính toán index price cho các index nhóm hiệu suất
industry_perform_index_dict = {}
industry_perform_index_price = pd.DataFrame()
for group, df in industry_perform.items():
    industry_perform_index_dict[group] = calculate_index(industry_perform, group)
    industry_perform_index_dict[group]['stock'] = group
    industry_perform_index_dict[group]['ma5_V'] = industry_perform_index_dict[group]['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    industry_perform_index_dict[group]['liquid_ratio'] = industry_perform_index_dict[group]['volume'] / (industry_perform_index_dict[group]['ma5_V'])
    industry_perform_index_dict[group]['liquid_ratio'].iloc[0] = industry_perform_index_dict[group]['volume'].iloc[0] / ((industry_perform_index_dict[group]['ma5_V']).iloc[0]*time_percent)
for index_name, df in industry_perform_index_dict.items():
    industry_perform_index_price = pd.concat([industry_perform_index_price, df.iloc[:60]], axis=0)

#Tính toán index price cho các index nhóm vốn hoá
marketcap_gorup_index_dict = {}
marketcap_group_index_price = pd.DataFrame()
for group, df in marketcap_group.items():
    marketcap_gorup_index_dict[group] = calculate_index(marketcap_group, group)
    marketcap_gorup_index_dict[group]['stock'] = group
    marketcap_gorup_index_dict[group]['ma5_V'] = marketcap_gorup_index_dict[group]['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    marketcap_gorup_index_dict[group]['liquid_ratio'] = marketcap_gorup_index_dict[group]['volume'] / (marketcap_gorup_index_dict[group]['ma5_V'])
    marketcap_gorup_index_dict[group]['liquid_ratio'].iloc[0] = marketcap_gorup_index_dict[group]['volume'].iloc[0] / ((marketcap_gorup_index_dict[group]['ma5_V']).iloc[0]*time_percent)
for index_name, df in marketcap_gorup_index_dict.items():
    marketcap_group_index_price = pd.concat([marketcap_group_index_price, df.iloc[:60]], axis=0)

#Thêm liquid_ratio vào các bảng trend
all_industry_perform_index_trend['liquid_ratio'] = industry_perform_index_price['liquid_ratio']
all_marketcap_group_index_trend['liquid_ratio'] = marketcap_group_index_price['liquid_ratio']
all_marketcap_group_index_trend['cap_order'] = all_marketcap_group_index_trend['id'].apply(lambda x: 0 if x=='LARGECAP' else (
                                                                                          1 if x=='MIDCAP' else(
                                                                                          2 if x=='SMALLCAP' else 3)))

t2m_index_trend = all_stock_trend['all_stock'].iloc[:60]
t2m_index_trend['liquid_ratio'] = all_stock_index_price['liquid_ratio']

In [45]:
all_stock_index_ta = {}
for index_name, df in all_stock_index_dict.items():
    all_stock_index_ta[index_name] = {}
    ta_result_list = concat_ta_df(df, 'index')
    all_stock_index_ta[index_name]['index_ta_df'] = ta_result_list[0]
    all_stock_index_ta[index_name]['index_ta_trend'] = ta_result_list[-1]

industry_index_ta = {}
for index_name, df in industry_index_dict.items():
    industry_index_ta[index_name] = {}
    ta_result_list = concat_ta_df(df, 'index')
    industry_index_ta[index_name]['index_ta_df'] = ta_result_list[0]
    industry_index_ta[index_name]['index_ta_trend'] = ta_result_list[-1]

industry_index_ta_df = pd.DataFrame()
industry_index_ta_trend = pd.DataFrame()
for index_name, value in industry_index_ta.items():
    industry_index_ta_df = pd.concat([industry_index_ta_df,value['index_ta_df']], axis=0)
    temp_df = value['index_ta_trend']
    temp_df['stock'] = index_name
    industry_index_ta_trend = pd.concat([industry_index_ta_trend,temp_df], axis=0)

industry_index_ta_df['stock'] = industry_index_ta_df['stock'].map(name_map_dict)
industry_index_ta_trend['stock'] = industry_index_ta_trend['stock'].map(name_map_dict)

In [46]:
#Tính toán index price cho VNINDEX
vnindex_price = index_dict['VNINDEX'].iloc[:60]
vnindex_price['color'] = vnindex_price.apply(lambda x: 'red' if x['close']<x['open'] else 'green', axis=1)
vnindex_ta_result_list = concat_ta_df(index_dict['VNINDEX'],'index')
vnindex_ta_df  = vnindex_ta_result_list[0]
vnindex_ta_trend = vnindex_ta_result_list[-1]

#Tính toán index price cho VN30F1M
vn30fu_price = index_fu_dict['VN30F1M'].iloc[:60]
vn30fu_price['color'] = vn30fu_price.apply(lambda x: 'red' if x['close']<x['open'] else 'green', axis=1)
vn30fu_ta_result_list = concat_ta_df(index_fu_dict['VN30F1M'],'index')
vn30fu_ta_df  = vn30fu_ta_result_list[0]
vn30fu_ta_trend = vn30fu_ta_result_list[-1]

#Gộp index price cho các index thị trường
for df, name in zip([vnindex_price, vn30fu_price, all_stock_index_price],['VNINDEX','VN30F1M','T2MINDEX']):
    df['index_name'] = name
for df, name in zip([vnindex_ta_df, vn30fu_ta_df, all_stock_index_ta['all_stock']['index_ta_df']],['VNINDEX','VN30F1M','T2MINDEX']):
    df['index_name'] = name
for df, name in zip([vnindex_ta_trend, vn30fu_ta_trend, all_stock_index_ta['all_stock']['index_ta_trend']],['VNINDEX','VN30F1M','T2MINDEX']):
    df['index_name'] = name

market_index_price = pd.concat([vnindex_price, vn30fu_price, all_stock_index_price], axis=0)[['stock','index_name','date','open','high','low','close','volume','color']]
market_index_ta_df = pd.concat([vnindex_ta_df, vn30fu_ta_df, all_stock_index_ta['all_stock']['index_ta_df']], axis=0)
market_index_ta_trend = pd.concat([vnindex_ta_trend, vn30fu_ta_trend, all_stock_index_ta['all_stock']['index_ta_trend']], axis=0)

#### Các bảng thay đổi diễn biến xếp hạng của các nhóm cổ phiếu

In [47]:
def create_ranking_df(score_df):
    socre_dict = {}
    for group in score_df.columns[1:-4]:
        socre_dict[group] = date_series.copy()
        socre_dict[group]['t0_score'] = score_df[group]
        socre_dict[group]['t5_score'] = socre_dict[group]['t0_score'][::-1].rolling(window=5).mean()[::-1]

    ranking_score = date_series.copy()
    for group in socre_dict.keys():
        ranking_score[group] = socre_dict[group]['t5_score']
        ranking_score.fillna(0,inplace=True)

    ranking_df = date_series.copy()
    for group in socre_dict.keys():
        ranking_df[group] = 0

    for i in range(len(date_series.copy())):
        ranking_df.iloc[i, 1:] = ranking_score.iloc[i, 1:].rank(ascending=False, method='min')
    
    ranking_df = ranking_df.head(20)

    return ranking_df

industry_ranking = create_ranking_df(industry_score_df)
industry_perform_ranking = create_ranking_df(industry_perform_score_df)
marketcap_group_ranking = create_ranking_df(marketcap_group_score_df)

industry_ranking_dict = {}
for column in industry_ranking.columns[1:]:
    industry_ranking_dict[column] = pd.DataFrame(industry_ranking['date'])
    industry_ranking_dict[column]['rank'] = industry_ranking[column]
    industry_ranking_dict[column]['id'] = column

industry_ranking_merge_df = pd.concat(industry_ranking_dict.values(),axis=0)
industry_ranking_merge_df['id'] = industry_ranking_merge_df['id'].map(name_map_dict)

In [48]:
#Bảng tổng hợp xếp hạng toàn bộ các nhóm ngành
score_df = industry_score_df
socre_dict = {}
for group in score_df.columns[1:-4]:
    socre_dict[group] = date_series.copy()
    socre_dict[group]['t0_score'] = score_df[group]
    socre_dict[group]['t5_score'] = socre_dict[group]['t0_score'][::-1].rolling(window=5).mean()[::-1]
ranking_score = date_series.copy()
for group in socre_dict.keys():
    ranking_score[group] = socre_dict[group]['t5_score']
    ranking_score.fillna(0,inplace=True)
ranking_df = date_series.copy()
for group in socre_dict.keys():
    ranking_df[group] = 0
for i in range(len(date_series.copy())):
    ranking_df.iloc[i, 1:] = ranking_score.iloc[i, 1:].rank(ascending=False, method='min')

industry_perform_dict = stock_classification.groupby('industry_name')['industry_perform'].first().to_dict()

score_arr = pd.DataFrame(ranking_score.iloc[0])
rank_arr = ranking_df.iloc[0]

industry_ranking_t0 = pd.concat([score_arr, rank_arr], axis=1).iloc[1:]
industry_ranking_t0.columns = ['score','rank']
industry_ranking_t0 = industry_ranking_t0.reset_index().rename(columns={'index':'industry'}).sort_values('rank')
industry_ranking_t0['industry_perform'] = industry_ranking_t0['industry'].map(industry_perform_dict)
industry_ranking_t0['industry'] = industry_ranking_t0['industry'].map(name_map_dict)
industry_ranking_t0['industry_perform'] = industry_ranking_t0['industry_perform'].map(name_map_dict)
industry_ranking_t0['rank'] = industry_ranking_t0['rank'].astype(int)

temp_df1 = industry_ranking_t0.iloc[0:6].reset_index(drop=True)
temp_df1.columns = ['industry_1', 'score_1', 'rank_1', 'industry_perform_1']
temp_df2 = industry_ranking_t0.iloc[6:12].reset_index(drop=True)
temp_df2.columns = ['industry_2', 'score_2', 'rank_2', 'industry_perform_2']
temp_df3 = industry_ranking_t0.iloc[12:18].reset_index(drop=True)
temp_df3.columns = ['industry_3', 'score_3', 'rank_3', 'industry_perform_3']
temp_df4 = industry_ranking_t0.iloc[18:24].reset_index(drop=True)
temp_df4.columns = ['industry_4', 'score_4', 'rank_4', 'industry_perform_4']

industry_ranking_t0 = pd.concat([temp_df1, temp_df2, temp_df3, temp_df4], axis=1)

#### Bảng dữ liệu top cổ phiếu trong vòng 5 phiên và phiên T0

In [49]:
#Biểu đồ cột chồng 5p các nhóm cổ phiếu
industry_perform_score_df_5p = industry_perform_score_df.iloc[:5,:-4]
industry_perform_score_df_5p['id'] = ['T-0','T-1','T-2','T-3','T-4']
industry_perform_score_df_5p = industry_perform_score_df_5p.drop(columns=['date'],axis=1).set_index('id').transpose().reset_index().rename(columns={'index':'name'})
industry_perform_score_df_5p['name'] = industry_perform_score_df_5p['name'].map(name_map_dict)

marketcap_group_score_df_5p = marketcap_group_score_df.iloc[:5,:-4]
marketcap_group_score_df_5p['id'] = ['T-0','T-1','T-2','T-3','T-4']
marketcap_group_score_df_5p = marketcap_group_score_df_5p.drop(columns=['date'],axis=1).set_index('id').transpose().reset_index().reset_index().rename(columns={'index':'name','level_0':'order'})
marketcap_group_score_df_5p['name'] = marketcap_group_score_df_5p['name'].map(name_map_dict)

industry_score_df_5p = industry_score_df.iloc[:5,:-4]
industry_score_df_5p['id'] = ['T-0','T-1','T-2','T-3','T-4']
industry_score_df_5p = industry_score_df_5p.drop(columns=['date'],axis=1).set_index('id').transpose().reset_index().rename(columns={'index':'name'})

perform_A_list = stock_classification[stock_classification['industry_perform']=='A']['industry_name'].unique()
perform_B_list = stock_classification[stock_classification['industry_perform']=='B']['industry_name'].unique()
perform_C_list = stock_classification[stock_classification['industry_perform']=='C']['industry_name'].unique()
perform_D_list = stock_classification[stock_classification['industry_perform']=='D']['industry_name'].unique()

perform_A = industry_score_df_5p[industry_score_df_5p['name'].isin(perform_A_list)].reset_index(drop=True).rename(columns={'name':'industry_1','T-0':'T0_1','T-1':'T1_1','T-2':'T2_1','T-3':'T3_1','T-4':'T4_1'})
perform_B = industry_score_df_5p[industry_score_df_5p['name'].isin(perform_B_list)].reset_index(drop=True).rename(columns={'name':'industry_2','T-0':'T0_2','T-1':'T1_2','T-2':'T2_2','T-3':'T3_2','T-4':'T4_2'})
perform_C = industry_score_df_5p[industry_score_df_5p['name'].isin(perform_C_list)].reset_index(drop=True).rename(columns={'name':'industry_3','T-0':'T0_3','T-1':'T1_3','T-2':'T2_3','T-3':'T3_3','T-4':'T4_3'})
perform_D = industry_score_df_5p[industry_score_df_5p['name'].isin(perform_D_list)].reset_index(drop=True).rename(columns={'name':'industry_4','T-0':'T0_4','T-1':'T1_4','T-2':'T2_4','T-3':'T3_4','T-4':'T4_4'})

industry_score_df_5p = pd.concat([perform_A,perform_B,perform_C,perform_D],axis=1)
industry_score_df_5p['industry_1'] = industry_score_df_5p['industry_1'].map(name_map_dict)
industry_score_df_5p['industry_2'] = industry_score_df_5p['industry_2'].map(name_map_dict)
industry_score_df_5p['industry_3'] = industry_score_df_5p['industry_3'].map(name_map_dict)
industry_score_df_5p['industry_4'] = industry_score_df_5p['industry_4'].map(name_map_dict)

In [50]:
#Tạo bảng full tất cả cổ phiếu trong 5 phiên
full_stock_score_dict = {}
for stock in final_score_dict.keys():
    full_stock_score_dict[stock] = final_score_dict[stock].iloc[:1]

full_stock_score_df = pd.concat(full_stock_score_dict.values(), axis=0)
full_stock_score_df['industry_name'] = full_stock_score_df['industry_name'].map(name_map_dict)
full_stock_score_df['industry_perform'] = full_stock_score_df['industry_perform'].map(name_map_dict)
full_stock_score_df['cap_order'] = full_stock_score_df['marketcap_group'].apply(lambda x: 0 if x=='large' else (
                                                                                          1 if x=='mid' else(
                                                                                          2 if x=='small' else 3)))
full_stock_score_df['marketcap_group'] = full_stock_score_df['marketcap_group'].map(name_map_dict)

full_stock_score_df['state_t0_score'] = full_stock_score_df['t0_score'].apply(lambda x: 'Tiền vào' if x>=0 else 'Tiền ra')
full_stock_score_df['state_t5_score'] = full_stock_score_df['t5_score'].apply(lambda x: 'Tiền vào' if x>=0 else 'Tiền ra')
full_stock_score_df['state_liquid_ratio'] = full_stock_score_df['liquid_ratio'].apply(lambda x: 'Rất thấp' if x<0.8 else 
                                                                                                ('Thấp' if (x>=0.8) & (x<0.95) else
                                                                                                ('Trung bình' if (x>=0.95) & (x<1.05) else
                                                                                                ('Cao' if (x>=1.05) & (x<1.2) else 'Rất cao'
                                                                                                ))))
full_stock_score_df['liquid_order'] = full_stock_score_df['state_liquid_ratio'].apply(lambda x: 5 if x=='Rất thấp' else 
                                                                                                (4 if x=='Thấp' else
                                                                                                (3 if x=='Trung bình' else
                                                                                                (2 if x=='Cao' else 1
                                                                                                ))))

In [51]:
#Top các cổ phiếu giao dịch tốt T0
top_all_stock_positive = full_stock_score_df[['stock','t0_score']].sort_values('t0_score', ascending=False)\
                        .rename(columns={'t0_score':'pos_score','stock':'pos_stock'}).head(10).reset_index(drop=True)
top_all_stock_negative = full_stock_score_df[['stock','t0_score']].sort_values('t0_score')\
                        .rename(columns={'t0_score':'neg_score','stock':'neg_stock'}).head(10).reset_index(drop=True)
top_all_stock_t0 = pd.concat([top_all_stock_positive,top_all_stock_negative], axis=1)

#### Bảng dữ liệu tổng hợp cho phần tra cứu và bộ lọc cổ phiếu

In [52]:
#Tạo bảng các chỉ số dòng tiền của cổ phiếu
temp_dict = copy.deepcopy({k: v for k, v in final_score_dict.items()})
for stock, df in temp_dict.items():
    df['score_change'] = (df['t0_score'][::-1].cumsum()[::-1] - df['t0_score'].iloc[-1])/100
    df['price_change'] = df['close'][::-1].pct_change()[::-1].fillna(0)[::-1].cumsum()[::-1]

stock_score_df_20p = pd.concat(temp_dict.values(), axis=0)
stock_score_df_20p['industry_name'] = stock_score_df_20p['industry_name'].map(name_map_dict)
stock_score_df_20p['industry_perform'] = stock_score_df_20p['industry_perform'].map(name_map_dict)
stock_score_df_20p['marketcap_group'] = stock_score_df_20p['marketcap_group'].map(name_map_dict)

#Tạo bảng dữ liêu biểu đồ giá và điểm dòng tiền
price_score_chart_20p = {}
for stock, df in final_score_dict.items():
    price_score_chart_20p[stock] = df.copy()
    price_score_chart_20p[stock]['stock'] = stock
    price_score_chart_20p[stock]['score_20p'] = (df['t0_score'][::-1].cumsum()[::-1] - df['t0_score'].iloc[-1].item())/100
    price_score_chart_20p[stock]['price_20p'] = df['price_change'][::-1].cumsum()[::-1] - df['price_change'].iloc[-1].item()
    price_score_chart_20p[stock].drop(columns=['industry_perform','industry_name','marketcap_group','rank','rank_change','t5_score','t0_score','price_change'], inplace=True)

price_score_chart_20p = pd.concat(price_score_chart_20p.values(), axis=0)

#Tạo bảng dữ liệu tổng hợp biểu đồ nến giá cổ phiếu
stock_price_chart_dict = {}
for stock in final_score_dict.keys():
    stock_price_chart_dict[stock] = stock_dict[stock][['date','open','high','low','close','volume']].head(60)
    stock_price_chart_dict[stock]['stock'] = stock
    stock_price_chart_dict[stock]['color'] = stock_price_chart_dict[stock].apply(lambda x: 'red' if x['close']<x['open'] else 'green', axis=1)
stock_price_chart = pd.concat(stock_price_chart_dict.values(), axis=0)

#Tạo các bảng dữ liệu xếp hạng các nhóm cổ phiếu melted
melted_industry_perform_ranking = pd.melt(industry_perform_ranking, id_vars=['date'], var_name='name', value_name='value')
melted_marketcap_group_ranking = pd.melt(marketcap_group_ranking, id_vars=['date'], var_name='name', value_name='value')
melted_industry_ranking = pd.melt(industry_ranking, id_vars=['date'], var_name='name', value_name='value')

melted_industry_perform_ranking['name'] = melted_industry_perform_ranking['name'].map(name_map_dict)
melted_marketcap_group_ranking['name'] = melted_marketcap_group_ranking['name'].map(name_map_dict)
melted_industry_ranking['name'] = melted_industry_ranking['name'].map(name_map_dict)

In [53]:
stock_dict_ta_df = {}
stock_dict_score_trend_df = {}

the_row_list = []

stock_candle_dict = {}
stock_pivot_dict = {}
stock_ma_dict = {}
stock_fibo_dict = {}

for stock in stock_classification_filtered['stock'].tolist():
    stock_ta_result_list = concat_ta_df(stock_dict[stock],'stock')
    stock_dict_ta_df[stock] = stock_ta_result_list[0]
    temp_row = stock_ta_result_list[-1]
    temp_row['stock'] = stock
    score_row = final_score_dict[stock][['stock','close','price_change','liquid_ratio','industry_name','industry_perform','marketcap_group','stock_perform','t2m_select','t0_score','t5_score','rank','rank_change']].iloc[:1]
    stock_dict_score_trend_df[stock] = pd.merge(score_row, temp_row, on='stock')

    stock_candle_dict[stock] = stock_ta_result_list[1][['stock','from_month_open','from_month_peak','from_month_bottom','from_quarter_open','from_quarter_peak','from_quarter_bottom','from_year_open','from_year_peak','from_year_bottom']].iloc[:10]
    stock_pivot_dict[stock] = stock_ta_result_list[2][['stock','from_month_pivot_p','from_quarter_pivot_p','from_year_pivot_p']].iloc[:10]
    stock_ma_dict[stock] = stock_ta_result_list[3][['stock','from_ma5','from_ma20','from_ma60','from_ma120','from_ma240','from_ma480']].iloc[:10]
    stock_fibo_dict[stock] = stock_ta_result_list[4][['stock','month_trend','from_month_fibo_382','from_month_fibo_500','from_month_fibo_618',
                                                        'quarter_trend','from_quarter_fibo_382','from_quarter_fibo_500','from_quarter_fibo_618',
                                                        'year_trend','from_year_fibo_382','from_year_fibo_500','from_year_fibo_618']].iloc[:10]
    
    temp_row_1 = stock_candle_dict[stock].iloc[:1]
    temp_row_2 = stock_pivot_dict[stock].iloc[:1]
    temp_row_3 = stock_ma_dict[stock].iloc[:1]
    temp_row_4 = stock_fibo_dict[stock].iloc[:1]
    temp_row_5 = final_score_dict[stock][['stock','close','price_change','liquid_ratio','industry_name','industry_perform','marketcap_group','stock_perform','t2m_select','t0_score','t5_score','rank','rank_change','top_count']].iloc[:1]
    temp_row_6 = stock_dict_score_trend_df[stock][['stock','rsi_state','month_price_trend','quarter_price_trend','year_price_trend']].iloc[:1]
    the_row = pd.merge(temp_row_1, pd.merge(temp_row_2, pd.merge(temp_row_3, pd.merge(temp_row_4, pd.merge(temp_row_5, temp_row_6, on='stock'), on='stock'), on='stock'), on='stock'), on='stock').iloc[0]
    the_row_list.append(the_row)

#Tạo các bảng chỉ số kĩ thuật cổ phiếu cho việc tra cứu
stock_ta_df = pd.concat(stock_dict_ta_df.values(), axis=0)
stock_score_trend_df = pd.concat(stock_dict_score_trend_df.values(), axis=0)
stock_score_trend_df['top_count'] = full_stock_score_df['top_count']
stock_score_trend_df['industry_name'] = stock_score_trend_df['industry_name'].map(name_map_dict)
stock_score_trend_df['marketcap_group'] = stock_score_trend_df['marketcap_group'].map(name_map_dict)
stock_score_trend_df['industry_perform'] = stock_score_trend_df['industry_perform'].map(name_map_dict)

#Tạo các bảng chỉ số kĩ thuật cổ phiếu cho việc tra cứu
stock_filter_df = pd.concat(the_row_list, axis=1).transpose().reset_index(drop=True)
stock_filter_df['t2m_select'] = stock_filter_df['t2m_select'].fillna('')
stock_filter_df['industry_name'] = stock_filter_df['industry_name'].map(name_map_dict)
stock_filter_df['marketcap_group'] = stock_filter_df['marketcap_group'].map(name_map_dict)
stock_filter_df['industry_perform'] = stock_filter_df['industry_perform'].map(name_map_dict)

stock_filter_df['state_t0_score'] = stock_filter_df['t0_score'].apply(lambda x: 'Tiền vào' if x>=0 else 'Tiền ra')
stock_filter_df['state_t5_score'] = stock_filter_df['t5_score'].apply(lambda x: 'Tiền vào' if x>=0 else 'Tiền ra')
stock_filter_df['state_liquid_ratio'] = stock_filter_df['liquid_ratio'].apply(lambda x: 'Rất thấp' if x<0.8 else 
                                                                                                ('Thấp' if (x>=0.8) & (x<0.95) else
                                                                                                ('Trung bình' if (x>=0.95) & (x<1.05) else
                                                                                                ('Cao' if (x>=1.05) & (x<1.2) else 'Rất cao'
                                                                                                ))))
stock_filter_df['liquid_order'] = stock_filter_df['state_liquid_ratio'].apply(lambda x: 5 if x=='Rất thấp' else 
                                                                                                (4 if x=='Thấp' else
                                                                                                (3 if x=='Trung bình' else
                                                                                                (2 if x=='Cao' else 1
                                                                                                ))))


temp_list = ['_month_open', '_month_peak', '_month_bottom','_quarter_open', '_quarter_peak', '_quarter_bottom',
            '_year_open', '_year_peak', '_year_bottom','_month_pivot_p', '_quarter_pivot_p', '_year_pivot_p',
            '_ma5', '_ma20', '_ma60', '_ma120', '_ma240','_ma480', '_month_fibo_382',
            '_month_fibo_500', '_month_fibo_618','_quarter_fibo_382', '_quarter_fibo_500',
            '_quarter_fibo_618', '_year_fibo_382','_year_fibo_500', '_year_fibo_618']

stock_ta_dict_count = {}
for stock in stock_classification_filtered['stock'].tolist():
    stock_ta_dict_count[stock] = pd.DataFrame()
    for temp_dict in [stock_candle_dict, stock_pivot_dict, stock_ma_dict, stock_fibo_dict]:
        for column in [item for item in temp_dict[stock].columns[1:] if 'trend' not in item]:
            stock_ta_dict_count[stock][f'count_{column}'] = temp_dict[stock][column][::-1].rolling(window=5, min_periods=1).apply(lambda x: (x > 0).sum(), raw=True)[::-1]

stock_ta_df_count = pd.DataFrame()
for stock, df in stock_ta_dict_count.items():
    stock_ta_df_count[stock] = df.iloc[0]
stock_ta_df_count = stock_ta_df_count.transpose().reset_index().rename(columns={'index':'stock'})

stock_filter_df = pd.merge(stock_filter_df, stock_ta_df_count, on='stock', how='left')
for name in temp_list:
    stock_filter_df[f'signal{name}'] = stock_filter_df.apply(lambda x: 
        'Tiệm cận kháng cự' if (x[f'from{name}']<0.03) & (x[f'from{name}']>-0.03) & (x[f'count_from{name}']<3) else 
        ('Tiệm cận hỗ trợ' if (x[f'from{name}']<0.03) & (x[f'from{name}']>-0.03) & (x[f'count_from{name}']>=3) else 'Không có tin hiệu'), axis=1)

for name in temp_list:
    stock_filter_df.drop(columns=[f'count_from{name}'], axis=1, inplace=True)

### 5.Lưu dữ liệu vào SQL Server

In [54]:
# now_df = pd.DataFrame([f"Cập nhật: {datetime.combine(date_series['date'].iloc[0].date(), current_time).strftime('%d/%m/%Y %H:%M:%S')}"])

# server = 'TWAN'
# database = 'MS'
# username = 'twan'
# password = 'chodom'
# engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server&trustServerCertificate=yes&charset=utf-8')

# #Xoá toàn bộ các bảng cũ
# conn = engine.connect()
# metadata = MetaData()
# metadata.reflect(bind=engine)
# for table in reversed(metadata.sorted_tables):
#     table.drop(engine)
# conn.close()

# # Lưu DataFrame vào SQL Server
# mareket_index_df.to_sql('p1_mareket_index_df', con=engine, if_exists='replace', index=False)
# liquidity_coef_df.to_sql('p1_liquidity_coef_df', con=engine, if_exists='replace', index=False)
# market_state_coef.to_sql('p1_market_state_coef', con=engine, if_exists='replace', index=False)
# fu_index_df.to_sql('p1_fu_index_df', con=engine, if_exists='replace', index=False)
# nn_td_top_stock.to_sql('p1_nn_td_top_stock', con=engine, if_exists='replace', index=False)
# nn_td_20p_df.to_sql('p1_nn_td_20p_df', con=engine, if_exists='replace', index=False)
# nn_td_buy_sell_df.to_sql('p1_nn_td_buy_sell_df', con=engine, if_exists='replace', index=False)
# top_all_stock_t0.to_sql('p1_top_all_stock_t0', con=engine, if_exists='replace', index=False)
# market_sentiment.to_sql('p1_market_sentiment', con=engine, if_exists='replace', index=False)
# market_breadth_flow_df.to_sql('p1_market_breadth_flow_df', con=engine, if_exists='replace', index=False)
# market_breadth_price_df.to_sql('p1_market_breadth_price_df', con=engine, if_exists='replace', index=False)
# market_index_price.to_sql('p1_market_index_price', con=engine, if_exists='replace', index=False)
# market_index_ta_df.to_sql('p1_market_index_ta_df', con=engine, if_exists='replace', index=False)
# market_index_ta_trend.to_sql('p1_market_index_ta_trend', con=engine, if_exists='replace', index=False)
# industry_name_liquid_df.to_sql('p1_industry_name_liquid_df', con=engine, if_exists='replace', index=False)
# industry_perform_liquid_df.to_sql('p1_industry_perform_liquid_df', con=engine, if_exists='replace', index=False)
# marketcap_group_liquid_df.to_sql('p1_marketcap_group_liquid_df', con=engine, if_exists='replace', index=False)
# market_breadth_liquid_df.to_sql('p1_market_breadth_liquid_df', con=engine, if_exists='replace', index=False)

# t2m_index_trend.to_sql('p2_all_stock_trend', con=engine, if_exists='replace', index=False)
# all_industry_perform_index_trend.to_sql('p2_all_industry_perform_index_trend', con=engine, if_exists='replace', index=False)
# all_marketcap_group_index_trend.to_sql('p2_all_marketcap_group_index_trend', con=engine, if_exists='replace', index=False)
# all_stock_score_month.to_sql('p2_all_stock_score_month', con=engine, if_exists='replace', index=False)
# all_stock_score_week.to_sql('p2_all_stock_score_week', con=engine, if_exists='replace', index=False)
# industry_perform_ranking.to_sql('p2_industry_perform_ranking', con=engine, if_exists='replace', index=False)
# marketcap_group_ranking.to_sql('p2_marketcap_group_ranking', con=engine, if_exists='replace', index=False)
# industry_perform_score_week.to_sql('p2_industry_perform_score_week', con=engine, if_exists='replace', index=False)
# industry_perform_score_month.to_sql('p2_industry_perform_score_month', con=engine, if_exists='replace', index=False)
# marketcap_group_score_week.to_sql('p2_marketcap_group_score_week', con=engine, if_exists='replace', index=False)
# marketcap_group_score_month.to_sql('p2_marketcap_group_score_month', con=engine, if_exists='replace', index=False)
# marketcap_group_score_df_5p.to_sql('p2_marketcap_group_score_df_5p', con=engine, if_exists='replace', index=False)
# industry_perform_score_df_5p.to_sql('p2_industry_perform_score_df_5p', con=engine, if_exists='replace', index=False)

# all_industry_index_price.to_sql('p3_all_industry_index_price', con=engine, if_exists='replace', index=False)
# all_industry_index_trend.to_sql('p3_all_industry_index_trend', con=engine, if_exists='replace', index=False)
# industry_ranking_t0.to_sql('p3_industry_ranking_t0', con=engine, if_exists='replace', index=False)
# industry_ranking.to_sql('p3_industry_ranking', con=engine, if_exists='replace', index=False)
# industry_score_week.to_sql('p3_industry_score_week', con=engine, if_exists='replace', index=False)
# industry_score_month.to_sql('p3_industry_score_month', con=engine, if_exists='replace', index=False)
# industry_index_ta_df.to_sql('p3_industry_index_ta_df', con=engine, if_exists='replace', index=False)
# industry_index_ta_trend.to_sql('p3_industry_index_ta_trend', con=engine, if_exists='replace', index=False)
# industry_ranking_merge_df.to_sql('p3_industry_ranking_merge_df', con=engine, if_exists='replace', index=False)

# stock_filter_df.to_sql('p4_stock_filter_df', con=engine, if_exists='replace', index=False)
# stock_score_df_20p.to_sql('p4_stock_score_df_20p', con=engine, if_exists='replace', index=False)
# stock_ta_df.to_sql('p4_stock_ta_df', con=engine, if_exists='replace', index=False)
# stock_score_trend_df.to_sql('p4_stock_score_trend_df', con=engine, if_exists='replace', index=False)
# stock_price_chart.to_sql('p4_stock_price_chart', con=engine, if_exists='replace', index=False)
# melted_industry_perform_ranking.to_sql('p4_melted_industry_perform_ranking', con=engine, if_exists='replace', index=False)
# melted_marketcap_group_ranking.to_sql('p4_melted_marketcap_group_ranking', con=engine, if_exists='replace', index=False)
# melted_industry_ranking.to_sql('p4_melted_industry_ranking', con=engine, if_exists='replace', index=False)

# now_df.to_sql('p0_now_df', con=engine, if_exists='replace', index=False)
# industry_score_df_5p.to_sql('p0_industry_score_df_5p', con=engine, if_exists='replace', index=False)
# marketcap_group_score_df_5p.to_sql('p0_marketcap_group_score_df_5p', con=engine, if_exists='replace', index=False)
# industry_perform_score_df_5p.to_sql('p0_industry_perform_score_df_5p', con=engine, if_exists='replace', index=False)
# full_stock_score_df.to_sql('p0_full_stock_score_df', con=engine, if_exists='replace', index=False)

# # Đóng kết nối
# engine.dispose()

In [55]:
# time_mark = now_df.iloc[0].item()[10:]
# today_date = date_series['date'].iloc[0].strftime("%d/%m/%Y")
# liquidity_coef = round(liquidity_coef_df['liquidity_coef'].item()*100, 2)
# market_state = market_state_coef['market_state'].item()
# sentiment_state = market_sentiment.iloc[0]['sentiment_state']
# vnindex_close = mareket_index_df[mareket_index_df['index']=='VNINDEX']['close'].item()
# vnindex_change_point = round(mareket_index_df[mareket_index_df['index']=='VNINDEX']['change_value'], 1).item()
# vnindex_change_percent = round(mareket_index_df[mareket_index_df['index']=='VNINDEX']['change_percent']*100, 2).item()
# market_volume = round(mareket_index_df[mareket_index_df['index']!='VN30']['value'].sum())
# nn_value = round(nn_td_buy_sell_df[nn_td_buy_sell_df['type'] == 'Mua-Bán']['GTGD_NN'].sum())

# tang_gia = int(market_breadth_price_df[market_breadth_price_df['name']=='Tăng giá']['all_stock_price_count'].item()*100/market_breadth_price_df['all_stock_price_count'].sum())
# khong_doi = int(market_breadth_price_df[market_breadth_price_df['name']=='Không đổi']['all_stock_price_count'].item()*100/market_breadth_price_df['all_stock_price_count'].sum())
# giam_gia = int(market_breadth_price_df[market_breadth_price_df['name']=='Giảm giá']['all_stock_price_count'].item()*100/market_breadth_price_df['all_stock_price_count'].sum())

# fu_index = round(fu_index_df[fu_index_df['index']=='VN30F1M']['close'],1).item()
# fu_change = round(fu_index_df[fu_index_df['index']=='VN30F1M']['change_value'],1).item()
# fu_volume = int(fu_index_df[fu_index_df['index']=='VN30F1M']['volume'].item())
# fu_nn_buy = int(fu_index_df[fu_index_df['index']=='VN30F1M']['nn_buy'].item())
# fu_nn_sell = int(fu_index_df[fu_index_df['index']=='VN30F1M']['nn_sell'].item())
# money_point_hsA = round(industry_perform_score_df_5p[industry_perform_score_df_5p['name']=='Hiệu suất A']['T-0'].item(),2)
# money_point_hsB = round(industry_perform_score_df_5p[industry_perform_score_df_5p['name']=='Hiệu suất B']['T-0'].item(),2)
# money_point_hsC = round(industry_perform_score_df_5p[industry_perform_score_df_5p['name']=='Hiệu suất C']['T-0'].item(),2)
# money_point_hsD = round(industry_perform_score_df_5p[industry_perform_score_df_5p['name']=='Hiệu suất D']['T-0'].item(),2)
# tienvao_hsA = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền vào']['A'].item()*100/market_breadth_flow_df['A'].sum())
# tienvao_hsB = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền vào']['B'].item()*100/market_breadth_flow_df['B'].sum())
# tienvao_hsC = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền vào']['C'].item()*100/market_breadth_flow_df['C'].sum())
# tienvao_hsD = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền vào']['D'].item()*100/market_breadth_flow_df['D'].sum())
# tienra_hsA = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền ra']['A'].item()*100/market_breadth_flow_df['A'].sum())
# tienra_hsB = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền ra']['B'].item()*100/market_breadth_flow_df['B'].sum())
# tienra_hsC = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền ra']['C'].item()*100/market_breadth_flow_df['C'].sum())
# tienra_hsD = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền ra']['D'].item()*100/market_breadth_flow_df['D'].sum())

# top1_hsA = industry_score_df_5p[['industry_1', 'T0_1']].sort_values('T0_1', ascending=False)['industry_1'].dropna().iloc[0]
# top2_hsA = industry_score_df_5p[['industry_1', 'T0_1']].sort_values('T0_1', ascending=False)['industry_1'].dropna().iloc[1]
# bot1_hsA = industry_score_df_5p[['industry_1', 'T0_1']].sort_values('T0_1', ascending=False)['industry_1'].dropna().iloc[-1]
# bot2_hsA = industry_score_df_5p[['industry_1', 'T0_1']].sort_values('T0_1', ascending=False)['industry_1'].dropna().iloc[-2]

# top1_hsB = industry_score_df_5p[['industry_2', 'T0_2']].sort_values('T0_2', ascending=False)['industry_2'].dropna().iloc[0]
# top2_hsB = industry_score_df_5p[['industry_2', 'T0_2']].sort_values('T0_2', ascending=False)['industry_2'].dropna().iloc[1]
# bot1_hsB = industry_score_df_5p[['industry_2', 'T0_2']].sort_values('T0_2', ascending=False)['industry_2'].dropna().iloc[-1]
# bot2_hsB = industry_score_df_5p[['industry_2', 'T0_2']].sort_values('T0_2', ascending=False)['industry_2'].dropna().iloc[-2]

# top1_hsC = industry_score_df_5p[['industry_3', 'T0_3']].sort_values('T0_3', ascending=False)['industry_3'].dropna().iloc[0]
# top2_hsC = industry_score_df_5p[['industry_3', 'T0_3']].sort_values('T0_3', ascending=False)['industry_3'].dropna().iloc[1]
# bot1_hsC = industry_score_df_5p[['industry_3', 'T0_3']].sort_values('T0_3', ascending=False)['industry_3'].dropna().iloc[-1]
# bot2_hsC = industry_score_df_5p[['industry_3', 'T0_3']].sort_values('T0_3', ascending=False)['industry_3'].dropna().iloc[-2]

# top1_hsD = industry_score_df_5p[['industry_4', 'T0_4']].sort_values('T0_4', ascending=False)['industry_4'].dropna().iloc[0]
# top2_hsD = industry_score_df_5p[['industry_4', 'T0_4']].sort_values('T0_4', ascending=False)['industry_4'].dropna().iloc[1]
# bot1_hsD = industry_score_df_5p[['industry_4', 'T0_4']].sort_values('T0_4', ascending=False)['industry_4'].dropna().iloc[-1]
# bot2_hsD = industry_score_df_5p[['industry_4', 'T0_4']].sort_values('T0_4', ascending=False)['industry_4'].dropna().iloc[-2]

# top_marketcap = marketcap_group_score_df_5p.sort_values('T-0').iloc[-1]['name']
# bot_marketcap = marketcap_group_score_df_5p.sort_values('T-0').iloc[0]['name']

# top_marketcap_point = round(marketcap_group_score_df_5p.sort_values('T-0').iloc[-1]['T-0'],2)
# bot_marketcap_point = round(marketcap_group_score_df_5p.sort_values('T-0').iloc[0]['T-0'],2)

# cap_dict = {
#     "LARGECAP": 'cap_large',
#     "MIDCAP": 'cap_mid',
#     "SMALLCAP": 'cap_small',
#     "PENNY": 'cap_penny'
# }
# tienvao_top_marketcap = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền vào'][cap_dict[top_marketcap]].item()*100/market_breadth_flow_df[cap_dict[top_marketcap]].sum())
# tienra_bot_marketcap = int(market_breadth_flow_df[market_breadth_flow_df['name']=='Tiền ra'][cap_dict[bot_marketcap]].item()*100/market_breadth_flow_df[cap_dict[bot_marketcap]].sum())

# top1_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[0]['stock']
# top1_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[0]['t0_score'],2)
# top1_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[0]['price_change']*100,2)

# top2_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[1]['stock']
# top2_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[1]['t0_score'],2)
# top2_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[1]['price_change']*100,2)

# top3_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[2]['stock']
# top3_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[2]['t0_score'],2)
# top3_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[2]['price_change']*100,2)

# top4_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[3]['stock']
# top4_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[3]['t0_score'],2)
# top4_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[3]['price_change']*100,2)

# top5_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[4]['stock']
# top5_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[4]['t0_score'],2)
# top5_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[4]['price_change']*100,2)

# bot1_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(0+1)]['stock']
# bot1_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(0+1)]['t0_score'],2)
# bot1_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(0+1)]['price_change']*100,2)

# bot2_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(1+1)]['stock']
# bot2_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(1+1)]['t0_score'],2)
# bot2_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(1+1)]['price_change']*100,2)

# bot3_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(2+1)]['stock']
# bot3_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(2+1)]['t0_score'],2)
# bot3_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(2+1)]['price_change']*100,2)

# bot4_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(3+1)]['stock']
# bot4_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(3+1)]['t0_score'],2)
# bot4_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(3+1)]['price_change']*100,2)

# bot5_stock = full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(4+1)]['stock']
# bot5_score = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(4+1)]['t0_score'],2)
# bot5_change = round(full_stock_score_df[['stock','t0_score','price_change']].sort_values('t0_score', ascending=False).iloc[-(4+1)]['price_change']*100,2)

# daily_report = f'''
# ({time_mark})
# Nhật ký thị trường phiên ngày {today_date}

# Chốt phiên ngày {today_date}, trạng thái dòng tiền ở mức {market_state}, chỉ số thanh khoản đạt {liquidity_coef}%, chỉ số tâm lý thị trường ở mức {sentiment_state}.

# VNINDEX đóng cửa ở mức {vnindex_close} ({vnindex_change_point}/{vnindex_change_percent}%) điểm, giá trị giao dịch đạt {market_volume} tỷ đồng, Net giao dịch khối ngoại đạt {nn_value} tỷ đồng. Thị trường có {tang_gia}% mã tăng giá, {giam_gia}% mã giảm giá và {khong_doi}% mã không đổi.
# Chỉ số phái sinh VN30F1M đóng cửa ở mức {fu_index} ({fu_change}) điểm,  khối lượng giao dịch đạt {fu_volume} hợp đồng. Khối ngoại giao dịch Mua/Bán là ({fu_nn_buy}/{fu_nn_sell}) hợp đồng.

# Dòng tiền nhóm Hiệu suất A ở mức {money_point_hsA} điểm, với {tienvao_hsA}% số cổ phiếu có tiền vào và {tienra_hsA}% có tiền ra. Trong đó, dòng tiền mạnh nhất ở nhóm ngành {top1_hsA}, {top2_hsA} và yếu nhất ở {bot1_hsA}, {bot2_hsA}
# Dòng tiền nhóm Hiệu suất B ở mức {money_point_hsB} điểm, với {tienvao_hsB}% số cổ phiếu có tiền vào và {tienra_hsB}% có tiền ra. Trong đó, dòng tiền mạnh nhất ở nhóm ngành {top1_hsB}, {top2_hsB} và yếu nhất ở {bot1_hsB}, {bot2_hsB}
# Dòng tiền nhóm Hiệu suất C ở mức {money_point_hsC} điểm, với {tienvao_hsC}% số cổ phiếu có tiền vào và {tienra_hsC}% có tiền ra. Trong đó, dòng tiền mạnh nhất ở nhóm ngành {top1_hsC}, {top2_hsC} và yếu nhất ở {bot1_hsC}, {bot2_hsC}
# Dòng tiền nhóm Hiệu suất D ở mức {money_point_hsD} điểm, với {tienvao_hsD}% số cổ phiếu có tiền vào và {tienra_hsD}% có tiền ra. Trong đó, dòng tiền mạnh nhất ở nhóm ngành {top1_hsD}, {top2_hsD} và yếu nhất ở {bot1_hsD}, {bot2_hsD}

# Nhóm vốn hoá {top_marketcap} có dòng tiền mạnh nhất ở mức {top_marketcap_point} điểm, với {tienvao_top_marketcap}% số cổ phiếu có tiền vào.
# Nhóm vốn hoá {bot_marketcap} có dòng tiền yếu nhất ở mức {bot_marketcap_point} điểm, với {tienra_bot_marketcap}% số cổ phiếu có tiền ra.

# Top 5 cổ phiếu có tiền vào tốt nhất thị trường: {top1_stock} ({top1_score}/{top1_change}%), {top2_stock} ({top2_score}/{top2_change}%), {top3_stock} ({top3_score}/{top3_change}%), {top4_stock} ({top4_score}/{top4_change}%), {top5_stock} ({top5_score}/{top5_change}%)
# Top 5 cổ phiếu có tiền ra nhiều nhất thị trường: {bot1_stock} ({bot1_score}/{bot1_change}%), {bot2_stock} ({bot2_score}/{bot2_change}%), {bot3_stock} ({bot3_score}/{bot3_change}%), {bot4_stock} ({bot4_score}/{bot4_change}%), {bot5_stock} ({bot5_score}/{bot5_change}%)

# #t2minvest
# Website: https://t2m.vn/
# >>> 🗒 Để sử dụng Hệ thống T2M Invest, bạn vui lòng đăng ký tại: https://forms.gle/9g84sq2mGspFzUxk7
# >>> ☘️ Để tham gia Cộng đồng T2M Invest, bạn vui lòng truy cập tại: https://t.me/+Ok-CFV91o7syZjU1
# '''

# with open("../daily_report.txt", 'w', encoding='utf-8') as file:
#     file.write(daily_report)

In [56]:
# username = 'twan'
# password = 'caythong5'
# database = 'ms'
# server = '192.168.9.132'
# engine = create_engine(f"mysql+pymysql://{username}:{password}@{server}/{database}")

# server = 'TWAN'
# database = 'ms'
# username = 'twan'
# password = 'chodom'
# engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server&trustServerCertificate=yes&charset=utf-8')